<a href="https://colab.research.google.com/github/rishabhraaj17/DeepLearningHW/blob/master/DeepMIL_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Import Packages

Chnage numpy version to be able to load dataset from .npy files

In [0]:
pip install numpy==1.16.1

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data_utils
import torch.optim as optim
from torch.autograd import Variable

torch.__version__

Using TensorFlow backend.


'1.1.0'

####Mount Google Drive 
Copy dataset files from Drive to current session

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
path = '/content/drive/My\ Drive'

####Train

In [0]:
!ls $path/Train

train_bag_0.npy		     train_bag_20000.npy
train_bag_100000_110000.npy  train_bag_210000_220000.npy
train_bag_110000_120000.npy  train_bag_220000_230000.npy
train_bag_120000_130000.npy  train_bag_230000_240000.npy
train_bag_130000_140000.npy  train_bag_240000_250000.npy
train_bag_140000_150000.npy  train_bag_250000_257573.npy
train_bag_150000_160000.npy  train_bag_30000_40000.npy
train_bag_160000_170000.npy  train_bag_40000_50000.npy
train_bag_170000_180000.npy  train_bag_50000_60000.npy
train_bag_180000_190000.npy  train_bag_60000_70000.npy
train_bag_190000_200000.npy  train_bag_70000_80000.npy
train_bag_200000_210000.npy  train_bag_80000_90000.npy
train_bag_20000_30000.npy    train_bag_90000_100000.npy


In [0]:
!cp $path/Train/train_bag* ./

Merge splits of dataset into one.

**emb** is short for *pre-extracted BERT embeddings.*
**lab** is short for *Bag level Lables*

For each of **Train**, **Test** and **Validation**

In [0]:
directory = './'
emb = list()
lab = list()

for file in os.listdir(directory):
     filename = os.fsdecode(file)
     if filename.endswith(".npy"):
        data = np.load(os.path.join(directory, filename))
        emb.append(data[0])
        lab.append(data[1])


emb_train = np.concatenate(emb, axis=0)
lab_train = np.concatenate(lab, axis=0)

In [0]:
del emb
del lab

####Test

In [0]:
!cp $path/Test/test_bag* ./

In [0]:
data = np.load('test_bag_0_50000.npy')

In [0]:
emb_test = data[0]
lab_test = data[1]

In [0]:
del data

####Validation

In [0]:
!cp $path/Val/val_bag* ./

In [0]:
data = np.load('val_bag_0_50000.npy')

In [0]:
emb_val = data[0]
lab_val = data[1]

### Tensorboard Logging

In [0]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

##Metrics

Used for evaluating models.

In [0]:
from sklearn import metrics

In [0]:
def evaluate_model(y_target, y_prediction):
    acc = metrics.accuracy_score(y_target, y_prediction)
    f1_micro = metrics.f1_score(y_target, y_prediction, average="micro")
    f1_macro = metrics.f1_score(y_target, y_prediction, average="macro")
    f1_weighted = metrics.f1_score(y_target, y_prediction, average="weighted")

    print((
        f"Accuracy: {acc}; \n"
        f"F1-Score (Micro): {f1_micro}; \n"
        f"F1-Score (Macro): {f1_macro}; \n"
        f"F1-Score (Weighted): {f1_weighted}"))

##MIL CNNs

In [0]:
class Attention(nn.Module):
    def __init__(self):
        super(Attention, self).__init__()
        self.L = 500
        self.D = 128
        self.K = 1

        self.feature_extractor_part1 = nn.Sequential(
            nn.Conv1d(768, 20, kernel_size=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv1d(10, 50, kernel_size=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2)
        )

        self.feature_extractor_part2 = nn.Sequential(
            nn.Linear(25 * 1, self.L),
            nn.ReLU(),
        )

        self.attention = nn.Sequential(
            nn.Linear(self.L, self.D),
            nn.Tanh(),
            nn.Linear(self.D, self.K)
        )

        self.classifier = nn.Sequential(
            nn.Linear(self.L*self.K, 3),
            nn.Softmax()
        )

    def forward(self, x):
        H = self.feature_extractor_part1(x)
        H = H.view(-1, 25 * 1)
        H = self.feature_extractor_part2(H)  # NxL

        A = self.attention(H)  # NxK
        A = torch.transpose(A, 1, 0)  # KxN
        A = F.softmax(A, dim=1)  # softmax over N

        M = torch.mm(A, H)  # KxL

        Y_prob = self.classifier(M)
        Y_hat = torch.argmax(Y_prob, dim=1).float()

        return Y_prob, Y_hat, A

In [0]:
seed = 5

if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    print('GPU is ON!')


GPU is ON!


In [0]:
model = Attention()

if torch.cuda.is_available():
    model.cuda()
    
print('Init Model')

Init Model


In [0]:
loss = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005, betas=(0.9, 0.999), weight_decay=10e-5)

In [0]:
cuda = True

In [0]:
running_loss = 0
total_train_loss = 0
i = 0
epochs = 5

####Train

In [0]:
for epoch in range(1, epochs + 1):
  i = 0
  for data, label in zip(emb_train, lab_train):
    i += 1
    data = torch.from_numpy(data)
    bag_label = label
            
    if cuda:
      data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
                
    d = data.reshape(*list(data.shape), 1)
    one_hot = F.one_hot(torch.tensor(bag_label), 3)
    data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))

    optimizer.zero_grad()

    # Forward pass, backward pass, optimize
    prob, hat, A = model.forward(data)
            
    loss_size = loss(prob, torch.argmax(bag_label,1))
    loss_size.backward()
    optimizer.step()

    running_loss = loss_size.data
    total_train_loss += loss_size.data
    accuracy = (torch.argmax(bag_label,1).to(torch.float) == hat.squeeze()).float().sum()
    
    if (i+1) % 10000 == 0:
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Acc: {:.2f}' 
               .format(epoch, epochs, i+1, len(emb_train), loss_size.item(), accuracy.item()))

        # ================================================================== #
        #                        Tensorboard Logging                         #
        # ================================================================== #

       # 1. Log scalar values (scalar summary)
        info = { 'loss': loss_size.item(), 'accuracy': accuracy.item() }

        for tag, value in info.items():
            writer.add_scalar(tag, value, i+1)

        # 2. Log values and gradients of the parameters (histogram summary)
        for tag, value in model.named_parameters():
            tag = tag.replace('.', '/')
            writer.add_histogram(tag, value.data.cpu().numpy(), i+1)
            writer.add_histogram(tag+'/grad', value.grad.data.cpu().numpy(), i+1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch [1/5], Step [10000/257353], Loss: 1.4607, Acc: 0.00
Epoch [1/5], Step [20000/257353], Loss: 1.3297, Acc: 0.00
Epoch [1/5], Step [30000/257353], Loss: 0.5809, Acc: 1.00
Epoch [1/5], Step [40000/257353], Loss: 0.5514, Acc: 1.00
Epoch [1/5], Step [50000/257353], Loss: 0.5574, Acc: 1.00
Epoch [1/5], Step [60000/257353], Loss: 0.5565, Acc: 1.00
Epoch [1/5], Step [70000/257353], Loss: 0.8517, Acc: 1.00
Epoch [1/5], Step [80000/257353], Loss: 0.6970, Acc: 1.00
Epoch [1/5], Step [90000/257353], Loss: 0.5515, Acc: 1.00
Epoch [1/5], Step [100000/257353], Loss: 0.5545, Acc: 1.00
Epoch [1/5], Step [110000/257353], Loss: 0.5562, Acc: 1.00
Epoch [1/5], Step [120000/257353], Loss: 0.5638, Acc: 1.00
Epoch [1/5], Step [130000/257353], Loss: 1.5257, Acc: 0.00
Epoch [1/5], Step [140000/257353], Loss: 0.5516, Acc: 1.00
Epoch [1/5], Step [150000/257353], Loss: 0.5520, Acc: 1.00
Epoch [1/5], Step [160000/257353], Loss: 0.6904, Acc: 1.00
Epoch [1/5], Step [170000/257353], Loss: 1.5412, Acc: 0.00
Epoch 

####Test

In [0]:
i = 0
correct = 0
y_target = list()
y_pred = list()
for data, label in zip(emb_test, lab_test):
  data = torch.from_numpy(data) # add
  bag_label = label
  if cuda:
    data, bag_label = data.cuda(), torch.tensor(bag_label).cuda() # update
  d = data.reshape(*list(data.shape), 1)
  one_hot = F.one_hot(torch.tensor(bag_label), 3)
  data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))
  
  prob, hat, A = model.forward(data)
  y_target.append(int(label))
  y_pred.append(hat.int().item())
  
  if int(hat.item()) == label:
    correct += 1
    
    
print((correct/len(lab_test)) * 100)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


71.336


#### Analyze

In [0]:
i = 0
correct = 0
for data, label in zip(emb_test, lab_test):
  data = torch.from_numpy(data) # add
  bag_label = label
  if cuda:
    data, bag_label = data.cuda(), torch.tensor(bag_label).cuda() # update
  d = data.reshape(*list(data.shape), 1)
  one_hot = F.one_hot(torch.tensor(bag_label), 3)
  data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))
  
  prob, hat, A = model.forward(data)
  print(i)
  print('Prob :', prob, '\nPredicted : ' , hat, '\nAttention : ',  A,'\nTarget : ', label, '\n*************************\n')
  
  i += 1
  if i == 50:
    break
  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


0
Prob : tensor([[9.9997e-01, 2.9058e-05, 4.8652e-07]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([0.], device='cuda:0') 
Attention :  tensor([[0.4858, 0.5142]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Target :  1 
*************************

1
Prob : tensor([[1.7476e-04, 9.9976e-01, 6.3243e-05]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([1.], device='cuda:0') 
Attention :  tensor([[0.1709, 0.1648, 0.1706, 0.1708, 0.1684, 0.1545]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Target :  1 
*************************

2
Prob : tensor([[7.5517e-02, 9.2448e-01, 2.8346e-16]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([1.], device='cuda:0') 
Attention :  tensor([[0.3338, 0.3338, 0.3325]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Target :  1 
*************************

3
Prob : tensor([[5.7650e-08, 6.8026e-02, 9.3197e-01]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  t

####Save

In [0]:
model_num = 0

In [0]:
PATH = 'model_state_'+str(model_num)
PATH0 = 'model_'+str(model_num)

torch.save(model.state_dict(), PATH)

torch.save(model, PATH0)

model_num += 1

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Attention. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


####Metrics

In [0]:
evaluate_model(y_target, y_pred)

Accuracy: 0.71336; 
F1-Score (Micro): 0.7133600000000001; 
F1-Score (Macro): 0.7172436349892788; 
F1-Score (Weighted): 0.718470922216232


##MIL FNNs

In [0]:
class AttentionFNN(nn.Module):
    def __init__(self):
        super(AttentionFNN, self).__init__()
        self.L = 500
        self.D = 128
        self.K = 1

        self.feature_extractor_part1 = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Linear(256, 32),
            nn.ReLU()
        )

        self.feature_extractor_part2 = nn.Sequential(
            nn.Linear(32, self.L),
            nn.ReLU(),
        )

        self.attention = nn.Sequential(
            nn.Linear(self.L, self.D),
            nn.Tanh(),
            nn.Linear(self.D, self.K)
        )

        self.classifier = nn.Sequential(
            nn.Linear(self.L*self.K, 3),
            nn.Softmax()
        )

    def forward(self, x):
        H = self.feature_extractor_part1(x)
        H = self.feature_extractor_part2(H)  # NxL

        A = self.attention(H)  # NxK
        A = torch.transpose(A, 1, 0)  # KxN
        A = F.softmax(A, dim=1)  # softmax over N

        M = torch.mm(A, H)  # KxL

        Y_prob = self.classifier(M)
        Y_hat = torch.argmax(Y_prob, dim=1).float()

        return Y_prob, Y_hat, A

In [0]:
seed = 5

if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    print('GPU is ON!')


GPU is ON!


In [0]:
model = AttentionFNN()
if torch.cuda.is_available():
    model.cuda()
    
print('Init Model')

Init Model


In [0]:
loss = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005, betas=(0.9, 0.999), weight_decay=10e-5)

In [0]:
cuda = True

In [0]:
running_loss = 0
total_train_loss = 0
i = 0
epochs = 5

####Train

In [0]:
for epoch in range(1, epochs + 1):
  i = 0
  for data, label in zip(emb_train, lab_train):
    i += 1
    data = torch.from_numpy(data)
    bag_label = label
            
    if cuda:
      data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
                
    d = data.reshape(*list(data.shape))
    one_hot = F.one_hot(torch.tensor(bag_label), 3)
    data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))

    optimizer.zero_grad()

    # Forward pass, backward pass, optimize
    prob, hat, A = model.forward(data)
            
    loss_size = loss(prob, torch.argmax(bag_label,1))
    loss_size.backward()
    optimizer.step()

    running_loss = loss_size.data
    total_train_loss += loss_size.data
    accuracy = (torch.argmax(bag_label,1).to(torch.float) == hat.squeeze()).float().sum()
    
    if (i+1) % 10000 == 0:
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Acc: {:.2f}' 
               .format(epoch, epochs, i+1, len(emb_train), loss_size.item(), accuracy.item()))

        # ================================================================== #
        #                        Tensorboard Logging                         #
        # ================================================================== #

       # 1. Log scalar values (scalar summary)
        info = { 'loss': loss_size.item(), 'accuracy': accuracy.item() }

        for tag, value in info.items():
            writer.add_scalar(tag, value, i+1)

        # 2. Log values and gradients of the parameters (histogram summary)
        for tag, value in model.named_parameters():
            tag = tag.replace('.', '/')
            writer.add_histogram(tag, value.data.cpu().numpy(), i+1)
            writer.add_histogram(tag+'/grad', value.grad.data.cpu().numpy(), i+1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch [1/5], Step [10000/257353], Loss: 1.5331, Acc: 0.00
Epoch [1/5], Step [20000/257353], Loss: 1.4092, Acc: 0.00
Epoch [1/5], Step [30000/257353], Loss: 1.5085, Acc: 0.00
Epoch [1/5], Step [40000/257353], Loss: 0.5514, Acc: 1.00
Epoch [1/5], Step [50000/257353], Loss: 0.5559, Acc: 1.00
Epoch [1/5], Step [60000/257353], Loss: 0.5536, Acc: 1.00
Epoch [1/5], Step [70000/257353], Loss: 1.1881, Acc: 0.00
Epoch [1/5], Step [80000/257353], Loss: 0.6007, Acc: 1.00
Epoch [1/5], Step [90000/257353], Loss: 0.5515, Acc: 1.00
Epoch [1/5], Step [100000/257353], Loss: 0.5541, Acc: 1.00
Epoch [1/5], Step [110000/257353], Loss: 0.5514, Acc: 1.00
Epoch [1/5], Step [120000/257353], Loss: 0.5515, Acc: 1.00
Epoch [1/5], Step [130000/257353], Loss: 1.2580, Acc: 0.00
Epoch [1/5], Step [140000/257353], Loss: 0.5515, Acc: 1.00
Epoch [1/5], Step [150000/257353], Loss: 0.5515, Acc: 1.00
Epoch [1/5], Step [160000/257353], Loss: 1.1627, Acc: 0.00
Epoch [1/5], Step [170000/257353], Loss: 1.5374, Acc: 0.00
Epoch 

####Test

In [0]:
i = 0
correct = 0
y_target = list()
y_pred = list()
for data, label in zip(emb_test, lab_test):
  data = torch.from_numpy(data)
  bag_label = label
  if cuda:
    data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
  d = data.reshape(*list(data.shape))
  one_hot = F.one_hot(torch.tensor(bag_label), 3)
  data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))

  prob, hat, A = model.forward(data)
  y_target.append(int(label))
  y_pred.append(hat.int().item())

  if int(hat.item()) == label:
    correct += 1
    
    
print((correct/len(lab_test)) * 100)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


69.134


#### Analyze

In [0]:
i = 0
correct = 0
for data, label in zip(emb_test, lab_test):
  data = torch.from_numpy(data)
  bag_label = label
  if cuda:
    data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
  d = data.reshape(*list(data.shape))
  one_hot = F.one_hot(torch.tensor(bag_label), 3)
  data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))
  
  prob, hat, A = model.forward(data)
  print(i)
  print('Prob :', prob, '\nPredicted : ' , hat, '\nAttention : ',  A,'\nTarget : ', label, '\n*************************\n')
  
  i += 1
  if i == 50:
    break
  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


0
Prob : tensor([[9.9862e-01, 7.5398e-04, 6.2402e-04]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([0.], device='cuda:0') 
Attention :  tensor([[0.5514, 0.4486]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Target :  1 
*************************

1
Prob : tensor([[4.9682e-05, 9.9995e-01, 8.6226e-07]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([1.], device='cuda:0') 
Attention :  tensor([[0.1415, 0.1924, 0.1415, 0.1415, 0.1497, 0.2335]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Target :  1 
*************************

2
Prob : tensor([[7.8800e-02, 9.2120e-01, 1.3247e-17]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([1.], device='cuda:0') 
Attention :  tensor([[0.3264, 0.3264, 0.3472]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Target :  1 
*************************

3
Prob : tensor([[1.7460e-06, 7.4849e-03, 9.9251e-01]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  t

####Save

In [0]:
PATH = 'modelFNN_state_'+str(model_num)
PATH0 = 'model_FNN_'+str(model_num)

torch.save(model.state_dict(), PATH)

torch.save(model, PATH0)

model_num += 1

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type AttentionFNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


####Metrics

In [0]:
evaluate_model(y_target, y_pred)

Accuracy: 0.69134; 
F1-Score (Micro): 0.69134; 
F1-Score (Macro): 0.6939714685692596; 
F1-Score (Weighted): 0.6954128021807596


##MIL FNNs with Dropout

In [0]:
class AttentionFNNDropout(nn.Module):
    def __init__(self):
        super(AttentionFNNDropout, self).__init__()
        self.L = 500
        self.D = 128
        self.K = 1

        self.feature_extractor_part1 = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, 32),
            nn.ReLU()
        )

        self.feature_extractor_part2 = nn.Sequential(
            nn.Linear(32, self.L),
            nn.ReLU(),
        )

        self.attention = nn.Sequential(
            nn.Linear(self.L, self.D),
            nn.Tanh(),
            nn.Linear(self.D, self.K)
        )

        self.classifier = nn.Sequential(
            nn.Linear(self.L*self.K, 3),
            nn.Softmax()
        )

    def forward(self, x):
        H = self.feature_extractor_part1(x)
        H = self.feature_extractor_part2(H)  # NxL

        A = self.attention(H)  # NxK
        A = torch.transpose(A, 1, 0)  # KxN
        A = F.softmax(A, dim=1)  # softmax over N

        M = torch.mm(A, H)  # KxL

        Y_prob = self.classifier(M)
        
        Y_hat = torch.argmax(Y_prob, dim=1).float()

        return Y_prob, Y_hat, A

In [0]:
seed = 5

if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    print('GPU is ON!')


GPU is ON!


In [0]:
model = AttentionFNNDropout()
if torch.cuda.is_available():
    model.cuda()
    
print('Init Model')

Init Model


In [0]:
loss = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005, betas=(0.9, 0.999), weight_decay=10e-5)

In [0]:
cuda = True

In [0]:
running_loss = 0
total_train_loss = 0
i = 0
epochs = 5

####Train

In [0]:
for epoch in range(1, epochs + 1):
  i = 0
  for data, label in zip(emb_train, lab_train):
    i += 1
    data = torch.from_numpy(data)
    bag_label = label
            
    if cuda:
      data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
                
    d = data.reshape(*list(data.shape))
    one_hot = F.one_hot(torch.tensor(bag_label), 3)
    data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))

    optimizer.zero_grad()

    # Forward pass, backward pass, optimize
    prob, hat, A = model.forward(data)
            
    loss_size = loss(prob, torch.argmax(bag_label,1))
    loss_size.backward()
    optimizer.step()

    running_loss = loss_size.data
    total_train_loss += loss_size.data
    accuracy = (torch.argmax(bag_label,1).to(torch.float) == hat.squeeze()).float().sum()
    
    if (i+1) % 10000 == 0:
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Acc: {:.2f}' 
               .format(epoch, epochs, i+1, len(emb_train), loss_size.item(), accuracy.item()))

        # ================================================================== #
        #                        Tensorboard Logging                         #
        # ================================================================== #

       # 1. Log scalar values (scalar summary)
        info = { 'loss': loss_size.item(), 'accuracy': accuracy.item() }

        for tag, value in info.items():
            writer.add_scalar(tag, value, i+1)

        # 2. Log values and gradients of the parameters (histogram summary)
        for tag, value in model.named_parameters():
            tag = tag.replace('.', '/')
            writer.add_histogram(tag, value.data.cpu().numpy(), i+1)
            writer.add_histogram(tag+'/grad', value.grad.data.cpu().numpy(), i+1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch [1/5], Step [10000/257353], Loss: 1.4770, Acc: 0.00
Epoch [1/5], Step [20000/257353], Loss: 0.9089, Acc: 1.00
Epoch [1/5], Step [30000/257353], Loss: 0.9226, Acc: 1.00
Epoch [1/5], Step [40000/257353], Loss: 0.5514, Acc: 1.00
Epoch [1/5], Step [50000/257353], Loss: 0.5652, Acc: 1.00
Epoch [1/5], Step [60000/257353], Loss: 0.5595, Acc: 1.00
Epoch [1/5], Step [70000/257353], Loss: 0.9238, Acc: 1.00
Epoch [1/5], Step [80000/257353], Loss: 0.8253, Acc: 1.00
Epoch [1/5], Step [90000/257353], Loss: 0.6351, Acc: 1.00
Epoch [1/5], Step [100000/257353], Loss: 1.1722, Acc: 0.00
Epoch [1/5], Step [110000/257353], Loss: 0.8628, Acc: 1.00
Epoch [1/5], Step [120000/257353], Loss: 0.5515, Acc: 1.00
Epoch [1/5], Step [130000/257353], Loss: 1.2916, Acc: 0.00
Epoch [1/5], Step [140000/257353], Loss: 0.5514, Acc: 1.00
Epoch [1/5], Step [150000/257353], Loss: 0.5524, Acc: 1.00
Epoch [1/5], Step [160000/257353], Loss: 0.5710, Acc: 1.00
Epoch [1/5], Step [170000/257353], Loss: 1.4904, Acc: 0.00
Epoch 

####Test

In [0]:
i = 0
correct = 0
y_target = list()
y_pred = list()
for data, label in zip(emb_test, lab_test):
  data = torch.from_numpy(data)
  bag_label = label
  if cuda:
    data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
  d = data.reshape(*list(data.shape))
  one_hot = F.one_hot(torch.tensor(bag_label), 3)
  data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))

  prob, hat, A = model.forward(data)
  y_target.append(int(label))
  y_pred.append(hat.int().item())

  if int(hat.item()) == label:
    correct += 1
    
    
print((correct/len(lab_test)) * 100)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


69.494


#### Analyze

In [0]:
i = 0
correct = 0
for data, label in zip(emb_test, lab_test):
  data = torch.from_numpy(data)
  bag_label = label
  if cuda:
    data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
  d = data.reshape(*list(data.shape))
  one_hot = F.one_hot(torch.tensor(bag_label), 3)
  data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))
  
  prob, hat, A = model.forward(data)
  print(i)
  print('Prob :', prob, '\nPredicted : ' , hat, '\nAttention : ',  A,'\nTarget : ', label, '\n*************************\n')
  
  i += 1
  if i == 50:
    break
  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


0
Prob : tensor([[0.9372, 0.0321, 0.0307]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([0.], device='cuda:0') 
Attention :  tensor([[0.5067, 0.4933]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Target :  1 
*************************

1
Prob : tensor([[6.7071e-04, 9.9724e-01, 2.0922e-03]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([1.], device='cuda:0') 
Attention :  tensor([[0.1647, 0.1712, 0.1651, 0.1647, 0.1650, 0.1693]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Target :  1 
*************************

2
Prob : tensor([[8.1138e-01, 1.8862e-01, 5.6226e-11]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([0.], device='cuda:0') 
Attention :  tensor([[0.3329, 0.3329, 0.3342]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Target :  1 
*************************

3
Prob : tensor([[1.5697e-10, 1.4347e-06, 1.0000e+00]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([2.], device=

####Save

In [0]:
PATH = 'modelFNNDropout_state_'+str(model_num)
PATH0 = 'model_Dropout_FNN_'+str(model_num)

torch.save(model.state_dict(), PATH)

torch.save(model, PATH0)

model_num += 1

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type AttentionFNNDropout. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


####Metrics

In [0]:
evaluate_model(y_target, y_pred)

Accuracy: 0.69494; 
F1-Score (Micro): 0.69494; 
F1-Score (Macro): 0.6877455857614011; 
F1-Score (Weighted): 0.6897591921896775


# Training for more Epochs

##MIL CNNs

In [0]:
seed = 5

if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    print('GPU is ON!')


GPU is ON!


In [0]:
model = Attention()
if torch.cuda.is_available():
    model.cuda()
    
print('Init Model')

Init Model


In [0]:
loss = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005, betas=(0.9, 0.999), weight_decay=10e-5)

In [0]:
cuda = True

In [0]:
running_loss = 0
total_train_loss = 0
i = 0
epochs = 25

####Train

In [0]:
for epoch in range(1, epochs + 1):
  i = 0
  for data, label in zip(emb_train, lab_train):
    i += 1
    data = torch.from_numpy(data)
    bag_label = label
            
    if cuda:
      data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
                
    d = data.reshape(*list(data.shape), 1)
    one_hot = F.one_hot(torch.tensor(bag_label), 3)
    data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))

    optimizer.zero_grad()

    # Forward pass, backward pass, optimize
    prob, hat, A = model.forward(data)
            
    loss_size = loss(prob, torch.argmax(bag_label,1))
    loss_size.backward()
    optimizer.step()

    running_loss = loss_size.data
    total_train_loss += loss_size.data
    accuracy = (torch.argmax(bag_label,1).to(torch.float) == hat.squeeze()).float().sum()
    
    if (i+1) % 10000 == 0:
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Acc: {:.2f}' 
               .format(epoch, epochs, i+1, len(emb_train), loss_size.item(), accuracy.item()))

        # ================================================================== #
        #                        Tensorboard Logging                         #
        # ================================================================== #

       # 1. Log scalar values (scalar summary)
        info = { 'loss': loss_size.item(), 'accuracy': accuracy.item() }

        for tag, value in info.items():
            writer.add_scalar(tag, value, i+1)

        # 2. Log values and gradients of the parameters (histogram summary)
        for tag, value in model.named_parameters():
            tag = tag.replace('.', '/')
            writer.add_histogram(tag, value.data.cpu().numpy(), i+1)
            writer.add_histogram(tag+'/grad', value.grad.data.cpu().numpy(), i+1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch [1/25], Step [10000/257353], Loss: 1.5238, Acc: 0.00
Epoch [1/25], Step [20000/257353], Loss: 1.4695, Acc: 0.00
Epoch [1/25], Step [30000/257353], Loss: 0.5774, Acc: 1.00
Epoch [1/25], Step [40000/257353], Loss: 0.5514, Acc: 1.00
Epoch [1/25], Step [50000/257353], Loss: 0.5827, Acc: 1.00
Epoch [1/25], Step [60000/257353], Loss: 0.5530, Acc: 1.00
Epoch [1/25], Step [70000/257353], Loss: 0.7761, Acc: 1.00
Epoch [1/25], Step [80000/257353], Loss: 0.5534, Acc: 1.00
Epoch [1/25], Step [90000/257353], Loss: 0.5514, Acc: 1.00
Epoch [1/25], Step [100000/257353], Loss: 0.5609, Acc: 1.00
Epoch [1/25], Step [110000/257353], Loss: 1.3425, Acc: 0.00
Epoch [1/25], Step [120000/257353], Loss: 0.5521, Acc: 1.00
Epoch [1/25], Step [130000/257353], Loss: 1.3984, Acc: 0.00
Epoch [1/25], Step [140000/257353], Loss: 0.5514, Acc: 1.00
Epoch [1/25], Step [150000/257353], Loss: 0.5520, Acc: 1.00
Epoch [1/25], Step [160000/257353], Loss: 1.1568, Acc: 0.00
Epoch [1/25], Step [170000/257353], Loss: 1.5414,

####Test

In [0]:
i = 0
correct = 0
y_target = list()
y_pred = list()
for data, label in zip(emb_test, lab_test):
  data = torch.from_numpy(data) 
  bag_label = label
  if cuda:
    data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
  d = data.reshape(*list(data.shape), 1)
  one_hot = F.one_hot(torch.tensor(bag_label), 3)
  data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))

  prob, hat, A = model.forward(data)
  y_target.append(int(label))
  y_pred.append(hat.int().item())

  if int(hat.item()) == label:
    correct += 1
    
    
print((correct/len(lab_test)) * 100)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


74.078


#### Analyze

In [0]:
i = 0
correct = 0
for data, label in zip(emb_test, lab_test):
  data = torch.from_numpy(data)
  bag_label = label
  if cuda:
    data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
  d = data.reshape(*list(data.shape), 1)
  one_hot = F.one_hot(torch.tensor(bag_label), 3)
  data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))

  prob, hat, A = model.forward(data)
  print(i)
  print('Prob :', prob, '\nPredicted : ' , hat, '\nAttention : ',  A,'\nTarget : ', label, '\n*************************\n')
  
  i += 1
  if i == 50:
    break
  

0
Prob : tensor([[0.9070, 0.0644, 0.0286]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([0.], device='cuda:0') 
Attention :  tensor([[0.4401, 0.5599]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Target :  1 
*************************

1
Prob : tensor([[6.2645e-07, 9.9934e-01, 6.5891e-04]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([1.], device='cuda:0') 
Attention :  tensor([[0.2015, 0.1554, 0.2010, 0.2010, 0.1975, 0.0436]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Target :  1 
*************************

2
Prob : tensor([[2.2847e-01, 7.7153e-01, 1.0482e-15]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([1.], device='cuda:0') 
Attention :  tensor([[0.3969, 0.3975, 0.2055]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Target :  1 
*************************

3
Prob : tensor([[8.7134e-13, 1.0988e-06, 1.0000e+00]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([2.], device=

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


 tensor([[0.2908, 0.3095, 0.3095, 0.0903]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Target :  2 
*************************

39
Prob : tensor([[9.9981e-01, 1.9080e-04, 2.1827e-06]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([0.], device='cuda:0') 
Attention :  tensor([[0.4637, 0.5363]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Target :  1 
*************************

40
Prob : tensor([[3.5104e-11, 5.2670e-07, 1.0000e+00]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([2.], device='cuda:0') 
Attention :  tensor([[0.1324, 0.0976, 0.1485, 0.1561, 0.1548, 0.0881, 0.1561, 0.0662]],
       device='cuda:0', grad_fn=<SoftmaxBackward>) 
Target :  2 
*************************

41
Prob : tensor([[2.7901e-06, 2.6695e-01, 7.3305e-01]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([2.], device='cuda:0') 
Attention :  tensor([[0.6552, 0.3448]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Target :  1 
**

####Save

In [0]:
model_num = 0

In [0]:
PATH = 'model_state_be'+str(model_num)
PATH0 = 'model_be'+str(model_num)

torch.save(model.state_dict(), PATH)

torch.save(model, PATH0)

model_num += 1

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Attention. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


####Metrics

In [0]:
evaluate_model(y_target, y_pred)

Accuracy: 0.74078; 
F1-Score (Micro): 0.74078; 
F1-Score (Macro): 0.7380059779498985; 
F1-Score (Weighted): 0.7393540803169427


##FNNs Dropout

In [0]:
seed = 5

if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    print('GPU is ON!')


GPU is ON!


In [0]:
model = AttentionFNNDropout()
if torch.cuda.is_available():
    model.cuda()
    
print('Init Model')

Init Model


In [0]:
loss = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005, betas=(0.9, 0.999), weight_decay=10e-5)

In [0]:
cuda = True

In [0]:
running_loss = 0
total_train_loss = 0
i = 0
epochs = 25

In [0]:
for epoch in range(1, epochs + 1):
  i = 0
  for data, label in zip(emb_train, lab_train):
    i += 1
    data = torch.from_numpy(data)
    bag_label = label
            
    if cuda:
      data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
                
    d = data.reshape(*list(data.shape))
    one_hot = F.one_hot(torch.tensor(bag_label), 3)
    data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))

    optimizer.zero_grad()

    # Forward pass, backward pass, optimize
    prob, hat, A = model.forward(data)
            
    loss_size = loss(prob, torch.argmax(bag_label,1))
    loss_size.backward()
    optimizer.step()

    running_loss = loss_size.data
    total_train_loss += loss_size.data
    accuracy = (torch.argmax(bag_label,1).to(torch.float) == hat.squeeze()).float().sum()
     
    if (i+1) % 10000 == 0:
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Acc: {:.2f}' 
               .format(epoch, epochs, i+1, len(emb_train), loss_size.item(), accuracy.item()))

        # ================================================================== #
        #                        Tensorboard Logging                         #
        # ================================================================== #

       # 1. Log scalar values (scalar summary)
        info = { 'loss': loss_size.item(), 'accuracy': accuracy.item() }

        for tag, value in info.items():
            writer.add_scalar(tag, value, i+1)

        # 2. Log values and gradients of the parameters (histogram summary)
        for tag, value in model.named_parameters():
            tag = tag.replace('.', '/')
            writer.add_histogram(tag, value.data.cpu().numpy(), i+1)
            writer.add_histogram(tag+'/grad', value.grad.data.cpu().numpy(), i+1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch [1/25], Step [10000/257353], Loss: 1.4590, Acc: 0.00
Epoch [1/25], Step [20000/257353], Loss: 1.4977, Acc: 0.00
Epoch [1/25], Step [30000/257353], Loss: 1.0014, Acc: 0.00
Epoch [1/25], Step [40000/257353], Loss: 0.5514, Acc: 1.00
Epoch [1/25], Step [50000/257353], Loss: 0.5785, Acc: 1.00
Epoch [1/25], Step [60000/257353], Loss: 0.5529, Acc: 1.00
Epoch [1/25], Step [70000/257353], Loss: 0.9225, Acc: 1.00
Epoch [1/25], Step [80000/257353], Loss: 0.7044, Acc: 1.00
Epoch [1/25], Step [90000/257353], Loss: 0.5514, Acc: 1.00
Epoch [1/25], Step [100000/257353], Loss: 0.5738, Acc: 1.00
Epoch [1/25], Step [110000/257353], Loss: 0.5880, Acc: 1.00
Epoch [1/25], Step [120000/257353], Loss: 0.5515, Acc: 1.00
Epoch [1/25], Step [130000/257353], Loss: 1.2140, Acc: 0.00
Epoch [1/25], Step [140000/257353], Loss: 0.5514, Acc: 1.00
Epoch [1/25], Step [150000/257353], Loss: 0.5516, Acc: 1.00
Epoch [1/25], Step [160000/257353], Loss: 0.8252, Acc: 1.00
Epoch [1/25], Step [170000/257353], Loss: 1.5192,

####Test

In [0]:
i = 0
correct = 0
y_target = list()
y_pred = list()
for data, label in zip(emb_test, lab_test):
  data = torch.from_numpy(data)
  bag_label = label
  if cuda:
    data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
  d = data.reshape(*list(data.shape))
  one_hot = F.one_hot(torch.tensor(bag_label), 3)
  data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))

  prob, hat, A = model.forward(data)
  y_target.append(int(label))
  y_pred.append(hat.int().item())

  if int(hat.item()) == label:
    correct += 1
    
    
print((correct/len(lab_test)) * 100)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


70.39999999999999


#### Analyze

In [0]:
i = 0
correct = 0
for data, label in zip(emb_test, lab_test):
  data = torch.from_numpy(data)
  bag_label = label
  if cuda:
    data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
  d = data.reshape(*list(data.shape))
  one_hot = F.one_hot(torch.tensor(bag_label), 3)
  data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))
 
  prob, hat, A = model.forward(data)
  print(i)
  print('Prob :', prob, '\nPredicted : ' , hat, '\nAttention : ',  A,'\nTarget : ', label, '\n*************************\n')
  
  i += 1
  if i == 50:
    break
  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


0
Prob : tensor([[9.9934e-01, 6.5722e-04, 5.4343e-06]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([0.], device='cuda:0') 
Attention :  tensor([[0.2101, 0.7899]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Target :  1 
*************************

1
Prob : tensor([[2.1187e-05, 9.9970e-01, 2.7887e-04]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([1.], device='cuda:0') 
Attention :  tensor([[0.1988, 0.1586, 0.1988, 0.1988, 0.1924, 0.0526]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Target :  1 
*************************

2
Prob : tensor([[4.3420e-01, 5.6580e-01, 3.9207e-17]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([1.], device='cuda:0') 
Attention :  tensor([[0.3341, 0.3341, 0.3318]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Target :  1 
*************************

3
Prob : tensor([[2.6412e-11, 7.4072e-06, 9.9999e-01]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  t

####Save

In [0]:
model_num = 0
PATH = 'modelFNNDropout_state25epoch_'+str(model_num)
PATH0 = 'model_Dropout_FNN_25epoch'+str(model_num)

torch.save(model.state_dict(), PATH)

torch.save(model, PATH0)

model_num += 1

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type AttentionFNNDropout. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


####Metrics

In [0]:
evaluate_model(y_target, y_pred)

Accuracy: 0.704; 
F1-Score (Micro): 0.704; 
F1-Score (Macro): 0.7025013749328766; 
F1-Score (Weighted): 0.704093438859345


##MIL CNNs 50 epochs

In [0]:
seed = 5

if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    print('GPU is ON!')


GPU is ON!


In [0]:
model = Attention()
if torch.cuda.is_available():
    model.cuda()
    
print('Init Model')

Init Model


In [0]:
loss = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005, betas=(0.9, 0.999), weight_decay=10e-5)

In [0]:
cuda = True

In [0]:
running_loss = 0
total_train_loss = 0
i = 0
epochs = 50

Checkpoint feature to be able to resume Training from a certain checkpoint. Since training for 25 epochs on Colab's GPU took more than 12 hours.

In [0]:
model_chk = './checkpoint'

####Train

In [0]:
model.train()

Attention(
  (feature_extractor_part1): Sequential(
    (0): Conv1d(768, 20, kernel_size=(1,), stride=(1,))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(10, 50, kernel_size=(1,), stride=(1,))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (feature_extractor_part2): Sequential(
    (0): Linear(in_features=25, out_features=500, bias=True)
    (1): ReLU()
  )
  (attention): Sequential(
    (0): Linear(in_features=500, out_features=128, bias=True)
    (1): Tanh()
    (2): Linear(in_features=128, out_features=1, bias=True)
  )
  (classifier): Sequential(
    (0): Linear(in_features=500, out_features=3, bias=True)
    (1): Softmax()
  )
)

In [0]:
for epoch in range(1, epochs + 1):
  i = 0
  for data, label in zip(emb_train, lab_train):
    i += 1
    data = torch.from_numpy(data)
    bag_label = label
            
    if cuda:
      data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
                
    d = data.reshape(*list(data.shape), 1)
    one_hot = F.one_hot(torch.tensor(bag_label), 3)
    data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))
    

    optimizer.zero_grad()

    # Forward pass, backward pass, optimize
    prob, hat, A = model.forward(data)
            
    loss_size = loss(prob, torch.argmax(bag_label,1))
    loss_size.backward()
    optimizer.step()

    running_loss = loss_size.data
    total_train_loss += loss_size.data
    accuracy = (torch.argmax(bag_label,1).to(torch.float) == hat.squeeze()).float().sum()
    
    if (i+1) % 10000 == 0:
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Acc: {:.2f}' 
               .format(epoch, epochs, i+1, len(emb_train), loss_size.item(), accuracy.item()))

        # ================================================================== #
        #                        Tensorboard Logging                         #
        # ================================================================== #

       # 1. Log scalar values (scalar summary)
        info = { 'loss': loss_size.item(), 'accuracy': accuracy.item() }

        for tag, value in info.items():
            writer.add_scalar(tag, value, i+1)

        # 2. Log values and gradients of the parameters (histogram summary)
        for tag, value in model.named_parameters():
            tag = tag.replace('.', '/')
            writer.add_histogram(tag, value.data.cpu().numpy(), i+1)
            writer.add_histogram(tag+'/grad', value.grad.data.cpu().numpy(), i+1)
            
            
  torch.save({
      'epoch': epoch,
      'model_state_dict': model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
      'loss': loss,
       }, model_chk+str(epoch)+'.pt')
  
  validate(emb_val, lab_val) # Validation after each epoch

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch [1/50], Step [10000/257353], Loss: 1.5507, Acc: 0.00
Epoch [1/50], Step [20000/257353], Loss: 0.7622, Acc: 1.00
Epoch [1/50], Step [30000/257353], Loss: 1.4583, Acc: 0.00
Epoch [1/50], Step [40000/257353], Loss: 0.5514, Acc: 1.00
Epoch [1/50], Step [50000/257353], Loss: 0.5658, Acc: 1.00
Epoch [1/50], Step [60000/257353], Loss: 0.5522, Acc: 1.00
Epoch [1/50], Step [70000/257353], Loss: 1.2660, Acc: 0.00
Epoch [1/50], Step [80000/257353], Loss: 0.5580, Acc: 1.00
Epoch [1/50], Step [90000/257353], Loss: 0.5514, Acc: 1.00
Epoch [1/50], Step [100000/257353], Loss: 0.5582, Acc: 1.00
Epoch [1/50], Step [110000/257353], Loss: 0.5630, Acc: 1.00
Epoch [1/50], Step [120000/257353], Loss: 0.5515, Acc: 1.00
Epoch [1/50], Step [130000/257353], Loss: 1.4219, Acc: 0.00
Epoch [1/50], Step [140000/257353], Loss: 0.5514, Acc: 1.00
Epoch [1/50], Step [150000/257353], Loss: 0.5514, Acc: 1.00
Epoch [1/50], Step [160000/257353], Loss: 0.8457, Acc: 1.00
Epoch [1/50], Step [170000/257353], Loss: 1.4774,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':


Validation Accuracy :  72.33800000000001
Epoch [2/50], Step [10000/257353], Loss: 1.4983, Acc: 0.00
Epoch [2/50], Step [20000/257353], Loss: 1.3266, Acc: 0.00
Epoch [2/50], Step [30000/257353], Loss: 1.5390, Acc: 0.00
Epoch [2/50], Step [40000/257353], Loss: 0.5514, Acc: 1.00
Epoch [2/50], Step [50000/257353], Loss: 0.5523, Acc: 1.00
Epoch [2/50], Step [60000/257353], Loss: 0.5518, Acc: 1.00
Epoch [2/50], Step [70000/257353], Loss: 1.4831, Acc: 0.00
Epoch [2/50], Step [80000/257353], Loss: 0.5584, Acc: 1.00
Epoch [2/50], Step [90000/257353], Loss: 0.5514, Acc: 1.00
Epoch [2/50], Step [100000/257353], Loss: 0.5517, Acc: 1.00
Epoch [2/50], Step [110000/257353], Loss: 0.5515, Acc: 1.00
Epoch [2/50], Step [120000/257353], Loss: 0.5514, Acc: 1.00
Epoch [2/50], Step [130000/257353], Loss: 1.4770, Acc: 0.00
Epoch [2/50], Step [140000/257353], Loss: 0.5514, Acc: 1.00
Epoch [2/50], Step [150000/257353], Loss: 0.5514, Acc: 1.00
Epoch [2/50], Step [160000/257353], Loss: 0.8841, Acc: 1.00
Epoch [2

#####Resume Training

Restore state dictionaries to be able to resume. 

In [0]:
checkpoint = torch.load('checkpoint43.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch_chkpoint = checkpoint['epoch']
loss = checkpoint['loss']

In [0]:
for epoch in range(epoch_chkpoint, epochs + 1):
  i = 0
  for data, label in zip(emb_train, lab_train):
    i += 1
    data = torch.from_numpy(data)
    bag_label = label
            
    if cuda:
      data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
                
    d = data.reshape(*list(data.shape), 1)
    one_hot = F.one_hot(torch.tensor(bag_label), 3)
    data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))
    

    optimizer.zero_grad()

    # Forward pass, backward pass, optimize
    prob, hat, A = model.forward(data)
            
    loss_size = loss(prob, torch.argmax(bag_label,1))
    loss_size.backward()
    optimizer.step()

    running_loss = loss_size.data
    total_train_loss += loss_size.data
    accuracy = (torch.argmax(bag_label,1).to(torch.float) == hat.squeeze()).float().sum()
    if (i+1) % 10000 == 0:
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Acc: {:.2f}' 
               .format(epoch, epochs, i+1, len(emb_train), loss_size.item(), accuracy.item()))

        # ================================================================== #
        #                        Tensorboard Logging                         #
        # ================================================================== #

       # 1. Log scalar values (scalar summary)
        info = { 'loss': loss_size.item(), 'accuracy': accuracy.item() }

        for tag, value in info.items():
            writer.add_scalar(tag, value, i+1)

        # 2. Log values and gradients of the parameters (histogram summary)
        for tag, value in model.named_parameters():
            tag = tag.replace('.', '/')
            writer.add_histogram(tag, value.data.cpu().numpy(), i+1)
            writer.add_histogram(tag+'/grad', value.grad.data.cpu().numpy(), i+1)
            
            
  torch.save({
      'epoch': epoch,
      'model_state_dict': model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
      'loss': loss,
       }, model_chk+str(epoch)+'.pt')
  
  validate(emb_val, lab_val) # Validation after each epoch

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch [43/50], Step [10000/257353], Loss: 1.5513, Acc: 0.00
Epoch [43/50], Step [20000/257353], Loss: 0.5525, Acc: 1.00
Epoch [43/50], Step [30000/257353], Loss: 0.9474, Acc: 1.00
Epoch [43/50], Step [40000/257353], Loss: 0.5514, Acc: 1.00
Epoch [43/50], Step [50000/257353], Loss: 0.5554, Acc: 1.00
Epoch [43/50], Step [60000/257353], Loss: 0.5515, Acc: 1.00
Epoch [43/50], Step [70000/257353], Loss: 1.5191, Acc: 0.00
Epoch [43/50], Step [80000/257353], Loss: 0.5515, Acc: 1.00
Epoch [43/50], Step [90000/257353], Loss: 0.5514, Acc: 1.00
Epoch [43/50], Step [100000/257353], Loss: 0.6179, Acc: 1.00
Epoch [43/50], Step [110000/257353], Loss: 0.5514, Acc: 1.00
Epoch [43/50], Step [120000/257353], Loss: 0.5514, Acc: 1.00
Epoch [43/50], Step [130000/257353], Loss: 0.7747, Acc: 1.00
Epoch [43/50], Step [140000/257353], Loss: 0.5514, Acc: 1.00
Epoch [43/50], Step [150000/257353], Loss: 0.5514, Acc: 1.00
Epoch [43/50], Step [160000/257353], Loss: 0.6611, Acc: 1.00
Epoch [43/50], Step [170000/25735

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':


Validation Accuracy :  70.25
Epoch [44/50], Step [10000/257353], Loss: 1.5498, Acc: 0.00
Epoch [44/50], Step [20000/257353], Loss: 0.5653, Acc: 1.00
Epoch [44/50], Step [30000/257353], Loss: 0.8900, Acc: 1.00
Epoch [44/50], Step [40000/257353], Loss: 0.5514, Acc: 1.00
Epoch [44/50], Step [50000/257353], Loss: 0.5523, Acc: 1.00
Epoch [44/50], Step [60000/257353], Loss: 0.5592, Acc: 1.00
Epoch [44/50], Step [70000/257353], Loss: 0.9073, Acc: 1.00
Epoch [44/50], Step [80000/257353], Loss: 0.5533, Acc: 1.00
Epoch [44/50], Step [90000/257353], Loss: 0.5514, Acc: 1.00
Epoch [44/50], Step [100000/257353], Loss: 0.5517, Acc: 1.00
Epoch [44/50], Step [110000/257353], Loss: 0.5515, Acc: 1.00
Epoch [44/50], Step [120000/257353], Loss: 0.5514, Acc: 1.00
Epoch [44/50], Step [130000/257353], Loss: 0.5631, Acc: 1.00
Epoch [44/50], Step [140000/257353], Loss: 0.5514, Acc: 1.00
Epoch [44/50], Step [150000/257353], Loss: 0.5514, Acc: 1.00
Epoch [44/50], Step [160000/257353], Loss: 0.7817, Acc: 1.00
Epoc

####Test

In [0]:
i = 0
correct = 0
y_target = list()
y_pred = list()
for data, label in zip(emb_test, lab_test):
  data = torch.from_numpy(data)
  bag_label = label
  if cuda:
    data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
  d = data.reshape(*list(data.shape), 1)
  one_hot = F.one_hot(torch.tensor(bag_label), 3)
  data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))
  
  prob, hat, A = model.forward(data)
  y_target.append(int(label))
  y_pred.append(hat.int().item())
  
  if int(hat.item()) == label:
    correct += 1
    
    
print((correct/len(lab_test)) * 100)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


73.088


####Validate

In [0]:
def validate(emb_val, lab_val):
  i = 0
  correct = 0
  y_target = list()
  y_pred = list()
  for data, label in zip(emb_val, lab_val):
    data = torch.from_numpy(data)
    bag_label = label
    if cuda:
      data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
    d = data.reshape(*list(data.shape), 1)
    one_hot = F.one_hot(torch.tensor(bag_label), 3)
    data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))
    
    prob, hat, A = model.forward(data)
    y_target.append(int(label))
    y_pred.append(hat.int().item())
    if int(hat.item()) == label:
      correct += 1


  print('Validation Accuracy : ', (correct/len(lab_val)) * 100)

#### Analyze

In [0]:
i = 0
correct = 0
for data, label in zip(emb_test, lab_test):
  data = torch.from_numpy(data)
  bag_label = label
  if cuda:
    data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
  d = data.reshape(*list(data.shape), 1)
  one_hot = F.one_hot(torch.tensor(bag_label), 3)
  data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))

  prob, hat, A = model.forward(data)
  print(i)
  print('Prob :', prob, '\nPredicted : ' , hat, '\nAttention : ',  A,'\nTarget : ', label, '\n*************************\n')
  
  i += 1
  if i == 50:
    break
  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


0
Prob : tensor([[9.9765e-01, 2.3454e-03, 8.8873e-06]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([0.], device='cuda:0') 
Attention :  tensor([[0.3797, 0.6203]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Target :  1 
*************************

1
Prob : tensor([[3.6781e-11, 9.9993e-01, 7.1167e-05]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([1.], device='cuda:0') 
Attention :  tensor([[0.1810, 0.1338, 0.1803, 0.1804, 0.1810, 0.1435]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Target :  1 
*************************

2
Prob : tensor([[2.0375e-04, 9.9980e-01, 1.3246e-23]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([1.], device='cuda:0') 
Attention :  tensor([[0.3463, 0.3463, 0.3074]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Target :  1 
*************************

3
Prob : tensor([[1.1957e-12, 1.1499e-02, 9.8850e-01]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  t

####Save

In [0]:
model_num = 0

In [0]:
PATH = 'model_state_CNN_50_Epochs'+str(model_num)+'.pt'
PATH0 = 'model__CNN_50_Epochs'+str(model_num)+'.pt'

torch.save(model.state_dict(), PATH)

torch.save(model, PATH0)

model_num += 1

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Attention. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


####Metrics

In [0]:
evaluate_model(y_target, y_pred)

Accuracy: 0.73088; 
F1-Score (Micro): 0.7308799999999999; 
F1-Score (Macro): 0.733557673398536; 
F1-Score (Weighted): 0.7346164932833777


###Fine-Tuning

####Loading Organic Dataset

Dataset is already pre-processed and converted into *Bert Embeddings and Bag Level Labels*

In [0]:
train = np.load('O_train_bag.npy')
test = np.load('O_test_bag.npy')
val = np.load('O_val_bag.npy')

In [0]:
emd_train = train[0]
lab_train = train[1]


emd_test = test[0]
lab_test = test[1]


emd_val = val[0]
lab_val = val[1]

In [0]:
model = torch.load('model__CNN_50_Epochs1.pt')

In [0]:
model.classifier

Sequential(
  (0): Linear(in_features=500, out_features=3, bias=True)
  (1): Softmax()
)

Load the pre-trained model on Amazon Fine Foods Dataset for fine-tuning it on Organic Dataset

In [0]:
organic_model = Attention()

In [0]:
organic_model.load_state_dict(model.state_dict(), strict=False)

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [0]:
seed = 5

if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    print('GPU is ON!')

    
if torch.cuda.is_available():
    organic_model.cuda()

loss = torch.nn.CrossEntropyLoss()

cuda = True

running_loss = 0
total_train_loss = 0
i = 0
epochs = 50

GPU is ON!


In [0]:
optimizer = optim.Adam(organic_model.parameters(), lr=0.005, betas=(0.9, 0.999), weight_decay=10e-5)

####Train

In [0]:
model_chk = './checkpoint'

In [0]:
organic_model.train()

Attention(
  (feature_extractor_part1): Sequential(
    (0): Conv1d(768, 20, kernel_size=(1,), stride=(1,))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(10, 50, kernel_size=(1,), stride=(1,))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (feature_extractor_part2): Sequential(
    (0): Linear(in_features=25, out_features=500, bias=True)
    (1): ReLU()
  )
  (attention): Sequential(
    (0): Linear(in_features=500, out_features=128, bias=True)
    (1): Tanh()
    (2): Linear(in_features=128, out_features=1, bias=True)
  )
  (classifier): Sequential(
    (0): Linear(in_features=500, out_features=3, bias=True)
    (1): Softmax()
  )
)

In [0]:
for epoch in range(1, epochs + 1):
  i = 0
  for data, label in zip(emd_train, lab_train):
    i += 1
    data = torch.from_numpy(data)
    bag_label = label
            
    if cuda:
      data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
                
    d = data.reshape(*list(data.shape), 1)
    one_hot = F.one_hot(torch.tensor(bag_label), 3)
    data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))
    

    optimizer.zero_grad()

    # Forward pass, backward pass, optimize
    prob, hat, A = organic_model.forward(data)
            
    loss_size = loss(prob, torch.argmax(bag_label,1))
    loss_size.backward()
    optimizer.step()

    running_loss = loss_size.data
    total_train_loss += loss_size.data
    accuracy = (torch.argmax(bag_label,1).to(torch.float) == hat.squeeze()).float().sum()
    
    if (i+1) % 1000 == 0:
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Acc: {:.2f}' 
               .format(epoch, epochs, i+1, len(emd_train), loss_size.item(), accuracy.item()))

        # ================================================================== #
        #                        Tensorboard Logging                         #
        # ================================================================== #

       # 1. Log scalar values (scalar summary)
        info = { 'loss': loss_size.item(), 'accuracy': accuracy.item() }

        for tag, value in info.items():
            writer.add_scalar(tag, value, i+1)

        # 2. Log values and gradients of the parameters (histogram summary)
        for tag, value in model.named_parameters():
            tag = tag.replace('.', '/')
            writer.add_histogram(tag, value.data.cpu().numpy(), i+1)
            # writer.add_histogram(tag+'/grad', value.grad.data.cpu().numpy(), i+1)
            
            
  torch.save({
      'epoch': epoch,
      'model_state_dict': model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
      'loss': loss,
       }, model_chk+str(epoch)+'.pt')
  
  validate(emd_val, lab_val)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch [1/50], Step [1000/4687], Loss: 1.5514, Acc: 0.00
Epoch [1/50], Step [2000/4687], Loss: 0.5514, Acc: 1.00
Epoch [1/50], Step [3000/4687], Loss: 0.5514, Acc: 1.00
Epoch [1/50], Step [4000/4687], Loss: 1.5514, Acc: 0.00


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':


Validation Accuracy :  30.676328502415455
Epoch [2/50], Step [1000/4687], Loss: 1.5514, Acc: 0.00
Epoch [2/50], Step [2000/4687], Loss: 0.5514, Acc: 1.00
Epoch [2/50], Step [3000/4687], Loss: 0.5514, Acc: 1.00
Epoch [2/50], Step [4000/4687], Loss: 0.5514, Acc: 1.00
Validation Accuracy :  29.71014492753623
Epoch [3/50], Step [1000/4687], Loss: 1.5514, Acc: 0.00
Epoch [3/50], Step [2000/4687], Loss: 0.5514, Acc: 1.00
Epoch [3/50], Step [3000/4687], Loss: 1.5514, Acc: 0.00
Epoch [3/50], Step [4000/4687], Loss: 0.5514, Acc: 1.00
Validation Accuracy :  39.61352657004831
Epoch [4/50], Step [1000/4687], Loss: 1.5514, Acc: 0.00
Epoch [4/50], Step [2000/4687], Loss: 1.5514, Acc: 0.00
Epoch [4/50], Step [3000/4687], Loss: 1.5514, Acc: 0.00
Epoch [4/50], Step [4000/4687], Loss: 0.5514, Acc: 1.00
Validation Accuracy :  39.61352657004831
Epoch [5/50], Step [1000/4687], Loss: 1.5514, Acc: 0.00
Epoch [5/50], Step [2000/4687], Loss: 1.5514, Acc: 0.00
Epoch [5/50], Step [3000/4687], Loss: 0.5514, Acc: 

####Validate

In [0]:
def validate(emb_val, lab_val):
  i = 0
  correct = 0
  y_target = list()
  y_pred = list()
  for data, label in zip(emb_val, lab_val):
    data = torch.from_numpy(data)
    bag_label = label
    if cuda:
      data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
    d = data.reshape(*list(data.shape), 1)
    one_hot = F.one_hot(torch.tensor(bag_label), 3)
    data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))
    
    prob, hat, A = organic_model.forward(data)
    y_target.append(int(label))
    y_pred.append(hat.int().item())
    if int(hat.item()) == label:
      correct += 1


  print('Validation Accuracy : ', (correct/len(lab_val)) * 100)

####Test

In [0]:
i = 0
correct = 0
y_target = list()
y_pred = list()
for data, label in zip(emd_test, lab_test):
  data = torch.from_numpy(data)
  bag_label = label
  if cuda:
    data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
  d = data.reshape(*list(data.shape), 1)
  one_hot = F.one_hot(torch.tensor(bag_label), 3)
  data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))
 
  prob, hat, A = organic_model.forward(data)
  y_target.append(int(label))
  y_pred.append(hat.int().item())

  if int(hat.item()) == label:
    correct += 1
    
    
print((correct/len(lab_test)) * 100)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


41.30434782608695


####Analyze

In [0]:
i = 0
correct = 0
for data, label in zip(emd_test, lab_test):
  data = torch.from_numpy(data)
  bag_label = label
  if cuda:
    data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
  d = data.reshape(*list(data.shape), 1)
  one_hot = F.one_hot(torch.tensor(bag_label), 3)
  data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))
 
  prob, hat, A = organic_model.forward(data)
  print(i)
  print('Prob :', prob, '\nPredicted : ' , hat, '\nAttention : ',  A,'\nTarget : ', label, '\n*************************\n')
  
  i += 1
  if i == 50:
    break

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


0
Prob : tensor([[9.2078e-06, 9.9999e-01, 2.4476e-06]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([1.], device='cuda:0') 
Attention :  tensor([[1.]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Target :  1 
*************************

1
Prob : tensor([[5.9808e-06, 9.9999e-01, 1.5863e-06]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([1.], device='cuda:0') 
Attention :  tensor([[1.]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Target :  1 
*************************

2
Prob : tensor([[2.5246e-06, 1.0000e+00, 6.4302e-07]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([1.], device='cuda:0') 
Attention :  tensor([[1.]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Target :  1 
*************************

3
Prob : tensor([[5.8604e-07, 1.0000e+00, 1.3563e-07]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([1.], device='cuda:0') 
Attention :  tensor([[1.]], device='cuda:0', grad_fn=

####Save

In [0]:
model_num = 0

In [0]:
PATH = 'organicTF50epoch'+str(model_num)+'.pt'
PATH0 = 'organicTF50_epoch_model'+str(model_num)+'.pt'

torch.save(model.state_dict(), PATH)

torch.save(model, PATH0)

model_num += 1

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Attention. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


####Evaluate

In [0]:
evaluate_model(y_target, y_pred)

Accuracy: 0.41304347826086957; 
F1-Score (Micro): 0.41304347826086957; 
F1-Score (Macro): 0.19487179487179487; 
F1-Score (Weighted): 0.24147157190635452


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


###Fine-Tuning with very low learning rate.

We saw that the fine-tuning did not went well with the learning rate we used for training the pre-trained model on Fine Food Dataset. Let us try to decrease the learning rate

In [0]:
model = torch.load('model__CNN_50_Epochs1.pt')

In [0]:
model.classifier

Sequential(
  (0): Linear(in_features=500, out_features=3, bias=True)
  (1): Softmax()
)

In [0]:
organic_model = Attention()

In [0]:
organic_model.load_state_dict(model.state_dict(), strict=False)

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [0]:
seed = 5

if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    print('GPU is ON!')

    
if torch.cuda.is_available():
    organic_model.cuda()

loss = torch.nn.CrossEntropyLoss()

cuda = True

running_loss = 0
total_train_loss = 0
i = 0
epochs = 50

GPU is ON!


In [0]:
optimizer = optim.Adam(organic_model.parameters(), lr=0.00005, betas=(0.9, 0.999), weight_decay=10e-5)

In [0]:
organic_model.train()

Attention(
  (feature_extractor_part1): Sequential(
    (0): Conv1d(768, 20, kernel_size=(1,), stride=(1,))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(10, 50, kernel_size=(1,), stride=(1,))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (feature_extractor_part2): Sequential(
    (0): Linear(in_features=25, out_features=500, bias=True)
    (1): ReLU()
  )
  (attention): Sequential(
    (0): Linear(in_features=500, out_features=128, bias=True)
    (1): Tanh()
    (2): Linear(in_features=128, out_features=1, bias=True)
  )
  (classifier): Sequential(
    (0): Linear(in_features=500, out_features=3, bias=True)
    (1): Softmax()
  )
)

####Train

In [0]:
for epoch in range(1, epochs + 1):
  i = 0
  for data, label in zip(emd_train, lab_train):
    i += 1
    data = torch.from_numpy(data)
    bag_label = label
            
    if cuda:
      data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
                
    d = data.reshape(*list(data.shape), 1)
    one_hot = F.one_hot(torch.tensor(bag_label), 3)
    data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))
    

    optimizer.zero_grad()

    # Forward pass, backward pass, optimize
    prob, hat, A = organic_model.forward(data)
            
    loss_size = loss(prob, torch.argmax(bag_label,1))
    loss_size.backward()
    optimizer.step()

    running_loss = loss_size.data
    total_train_loss += loss_size.data
    accuracy = (torch.argmax(bag_label,1).to(torch.float) == hat.squeeze()).float().sum()
    
    if (i+1) % 1000 == 0:
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Acc: {:.2f}' 
               .format(epoch, epochs, i+1, len(emd_train), loss_size.item(), accuracy.item()))

        # ================================================================== #
        #                        Tensorboard Logging                         #
        # ================================================================== #

       # 1. Log scalar values (scalar summary)
        info = { 'loss': loss_size.item(), 'accuracy': accuracy.item() }

        for tag, value in info.items():
            writer.add_scalar(tag, value, i+1)

        # 2. Log values and gradients of the parameters (histogram summary)
        for tag, value in model.named_parameters():
            tag = tag.replace('.', '/')
            writer.add_histogram(tag, value.data.cpu().numpy(), i+1)
            # writer.add_histogram(tag+'/grad', value.grad.data.cpu().numpy(), i+1)
            
            
  torch.save({
      'epoch': epoch,
      'model_state_dict': model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
      'loss': loss,
       }, model_chk+str(epoch)+'.pt')
  
  validate(emd_val, lab_val)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch [1/50], Step [1000/4687], Loss: 1.5502, Acc: 0.00
Epoch [1/50], Step [2000/4687], Loss: 0.9052, Acc: 1.00
Epoch [1/50], Step [3000/4687], Loss: 0.5632, Acc: 1.00
Epoch [1/50], Step [4000/4687], Loss: 1.5272, Acc: 0.00


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':


Validation Accuracy :  49.51690821256038
Epoch [2/50], Step [1000/4687], Loss: 1.4746, Acc: 0.00
Epoch [2/50], Step [2000/4687], Loss: 1.1743, Acc: 0.00
Epoch [2/50], Step [3000/4687], Loss: 0.5525, Acc: 1.00
Epoch [2/50], Step [4000/4687], Loss: 1.0564, Acc: 0.00
Validation Accuracy :  50.48309178743962
Epoch [3/50], Step [1000/4687], Loss: 0.8718, Acc: 1.00
Epoch [3/50], Step [2000/4687], Loss: 1.4813, Acc: 0.00
Epoch [3/50], Step [3000/4687], Loss: 0.5747, Acc: 1.00
Epoch [3/50], Step [4000/4687], Loss: 0.5742, Acc: 1.00
Validation Accuracy :  51.20772946859904
Epoch [4/50], Step [1000/4687], Loss: 0.6116, Acc: 1.00
Epoch [4/50], Step [2000/4687], Loss: 1.5128, Acc: 0.00
Epoch [4/50], Step [3000/4687], Loss: 0.5817, Acc: 1.00
Epoch [4/50], Step [4000/4687], Loss: 0.5957, Acc: 1.00
Validation Accuracy :  52.89855072463768
Epoch [5/50], Step [1000/4687], Loss: 0.5844, Acc: 1.00
Epoch [5/50], Step [2000/4687], Loss: 1.5066, Acc: 0.00
Epoch [5/50], Step [3000/4687], Loss: 0.6341, Acc: 1

####Validate

In [0]:
def validate(emb_val, lab_val):
  i = 0
  correct = 0
  y_target = list()
  y_pred = list()
  for data, label in zip(emb_val, lab_val):
    data = torch.from_numpy(data)
    bag_label = label
    if cuda:
      data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
    d = data.reshape(*list(data.shape), 1)
    one_hot = F.one_hot(torch.tensor(bag_label), 3)
    data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))
    
    prob, hat, A = organic_model.forward(data)
    y_target.append(int(label))
    y_pred.append(hat.int().item())
    if int(hat.item()) == label:
      correct += 1


  print('Validation Accuracy : ', (correct/len(lab_val)) * 100)

####Test

In [0]:
i = 0
correct = 0
y_target = list()
y_pred = list()
for data, label in zip(emd_test, lab_test):
  data = torch.from_numpy(data) 
  bag_label = label
  if cuda:
    data, bag_label = data.cuda(), torch.tensor(bag_label).cuda() 
  d = data.reshape(*list(data.shape), 1)
  one_hot = F.one_hot(torch.tensor(bag_label), 3)
  data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))
  
  prob, hat, A = organic_model.forward(data)
  y_target.append(int(label))
  y_pred.append(hat.int().item())
 
  if int(hat.item()) == label:
    correct += 1
    
    
print((correct/len(lab_test)) * 100)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


56.739130434782616


Decreasing the learning rate helped!

####Analyze

In [0]:
i = 0
correct = 0
for data, label in zip(emd_test, lab_test):
  data = torch.from_numpy(data)
  bag_label = label
  if cuda:
    data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
  d = data.reshape(*list(data.shape), 1)
  one_hot = F.one_hot(torch.tensor(bag_label), 3)
  data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))
  
  prob, hat, A = organic_model.forward(data)
  print(i)
  print('Prob :', prob, '\nPredicted : ' , hat, '\nAttention : ',  A,'\nTarget : ', label, '\n*************************\n')
  
  i += 1
  if i == 50:
    break

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


0
Prob : tensor([[1.1206e-08, 1.0000e+00, 6.1858e-08]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([1.], device='cuda:0') 
Attention :  tensor([[1.]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Target :  1 
*************************

1
Prob : tensor([[1.0243e-10, 1.0000e+00, 2.4777e-12]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([1.], device='cuda:0') 
Attention :  tensor([[1.]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Target :  1 
*************************

2
Prob : tensor([[4.3828e-06, 1.0000e+00, 1.5409e-13]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([1.], device='cuda:0') 
Attention :  tensor([[1.]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Target :  1 
*************************

3
Prob : tensor([[9.9527e-01, 4.4906e-03, 2.4053e-04]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([0.], device='cuda:0') 
Attention :  tensor([[1.]], device='cuda:0', grad_fn=

####Save

In [0]:
PATH = 'organicTF50epoch'+str(model_num)+'.pt'
PATH0 = 'organicTF50_epoch_model'+str(model_num)+'.pt'

torch.save(model.state_dict(), PATH)

torch.save(model, PATH0)

model_num += 1

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Attention. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


####Evaluate

In [0]:
evaluate_model(y_target, y_pred)

Accuracy: 0.5673913043478261; 
F1-Score (Micro): 0.5673913043478261; 
F1-Score (Macro): 0.5396239625073823; 
F1-Score (Weighted): 0.5533975044542362


##MIL CNN with 1D MaxPool

In [0]:
class Attention(nn.Module):
    def __init__(self):
        super(Attention, self).__init__()
        self.L = 500
        self.D = 128
        self.K = 1

        self.feature_extractor_part1 = nn.Sequential(
            nn.Conv1d(768, 20, kernel_size=1),
            nn.ReLU(),
            nn.MaxPool1d(2, stride=2),
            nn.Conv1d(20, 50, kernel_size=1),
            nn.ReLU(),
            nn.MaxPool1d(2, stride=2)
        )

        self.feature_extractor_part2 = nn.Sequential(
            nn.Linear(50 * 1, self.L),
            nn.ReLU(),
        )

        self.attention = nn.Sequential(
            nn.Linear(self.L, self.D),
            nn.Tanh(),
            nn.Linear(self.D, self.K)
        )

        self.classifier = nn.Sequential(
            nn.Linear(self.L*self.K, 3),
            nn.Softmax()
        )

    def forward(self, x):
        H = self.feature_extractor_part1(x)
        H = H.view(-1, 50 * 1)
        H = self.feature_extractor_part2(H)  # NxL

        A = self.attention(H)  # NxK
        A = torch.transpose(A, 1, 0)  # KxN
        A = F.softmax(A, dim=1)  # softmax over N

        M = torch.mm(A, H)  # KxL

        Y_prob = self.classifier(M)
        
        Y_hat = torch.argmax(Y_prob, dim=1).float()

        return Y_prob, Y_hat, A

In [0]:
seed = 5

if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    print('GPU is ON!')


GPU is ON!


In [0]:
model = Attention()
if torch.cuda.is_available():
    model.cuda()
    
print('Init Model')

Init Model


In [0]:
loss = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005, betas=(0.9, 0.999), weight_decay=10e-5)

In [0]:
cuda = True

In [0]:
running_loss = 0
total_train_loss = 0
i = 0
epochs = 20

Checkpoint saving to be able to resume training.

In [0]:
model_chk = './checkpoint'

####Train

In [0]:
model.train()

Attention(
  (feature_extractor_part1): Sequential(
    (0): Conv1d(768, 20, kernel_size=(1,), stride=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(20, 50, kernel_size=(1,), stride=(1,))
    (4): ReLU()
    (5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (feature_extractor_part2): Sequential(
    (0): Linear(in_features=50, out_features=500, bias=True)
    (1): ReLU()
  )
  (attention): Sequential(
    (0): Linear(in_features=500, out_features=128, bias=True)
    (1): Tanh()
    (2): Linear(in_features=128, out_features=1, bias=True)
  )
  (classifier): Sequential(
    (0): Linear(in_features=500, out_features=3, bias=True)
    (1): Softmax()
  )
)

In [0]:
for epoch in range(1, epochs + 1):
  i = 0
  for data, label in zip(emb_train, lab_train):
    i += 1
    data = torch.from_numpy(data)
    bag_label = label
            
    if cuda:
      data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
                
    d = data.reshape(*list(data.shape), 1)
    one_hot = F.one_hot(torch.tensor(bag_label), 3)
    data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))
    

    optimizer.zero_grad()

    # Forward pass, backward pass, optimize
    prob, hat, A = model.forward(data)
            
    loss_size = loss(prob, torch.argmax(bag_label,1))
    loss_size.backward()
    optimizer.step()

    running_loss = loss_size.data
    total_train_loss += loss_size.data
    accuracy = (torch.argmax(bag_label,1).to(torch.float) == hat.squeeze()).float().sum()
    
    if (i+1) % 10000 == 0:
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Acc: {:.2f}' 
               .format(epoch, epochs, i+1, len(emb_train), loss_size.item(), accuracy.item()))

        # ================================================================== #
        #                        Tensorboard Logging                         #
        # ================================================================== #

       # 1. Log scalar values (scalar summary)
        info = { 'loss': loss_size.item(), 'accuracy': accuracy.item() }

        for tag, value in info.items():
            writer.add_scalar(tag, value, i+1)

        # 2. Log values and gradients of the parameters (histogram summary)
        for tag, value in model.named_parameters():
            tag = tag.replace('.', '/')
            writer.add_histogram(tag, value.data.cpu().numpy(), i+1)
            writer.add_histogram(tag+'/grad', value.grad.data.cpu().numpy(), i+1)
            
            
  torch.save({
      'epoch': epoch,
      'model_state_dict': model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
      'loss': loss,
       }, model_chk+str(epoch)+'.pt')
  
  validate(emb_val, lab_val) # Validation after each Epoch

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch [1/20], Step [10000/257353], Loss: 1.5475, Acc: 0.00
Epoch [1/20], Step [20000/257353], Loss: 0.6236, Acc: 1.00
Epoch [1/20], Step [30000/257353], Loss: 1.4586, Acc: 0.00
Epoch [1/20], Step [40000/257353], Loss: 1.1600, Acc: 0.00
Epoch [1/20], Step [50000/257353], Loss: 0.5756, Acc: 1.00
Epoch [1/20], Step [60000/257353], Loss: 0.5514, Acc: 1.00
Epoch [1/20], Step [70000/257353], Loss: 1.5480, Acc: 0.00
Epoch [1/20], Step [80000/257353], Loss: 1.5224, Acc: 0.00
Epoch [1/20], Step [90000/257353], Loss: 1.5512, Acc: 0.00
Epoch [1/20], Step [100000/257353], Loss: 1.5489, Acc: 0.00
Epoch [1/20], Step [110000/257353], Loss: 1.5514, Acc: 0.00
Epoch [1/20], Step [120000/257353], Loss: 0.5514, Acc: 1.00
Epoch [1/20], Step [130000/257353], Loss: 1.4822, Acc: 0.00
Epoch [1/20], Step [140000/257353], Loss: 0.5514, Acc: 1.00
Epoch [1/20], Step [150000/257353], Loss: 0.5533, Acc: 1.00
Epoch [1/20], Step [160000/257353], Loss: 0.5514, Acc: 1.00
Epoch [1/20], Step [170000/257353], Loss: 0.5619,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':


Validation Accuracy :  70.666
Epoch [2/20], Step [10000/257353], Loss: 1.5472, Acc: 0.00
Epoch [2/20], Step [20000/257353], Loss: 0.5892, Acc: 1.00
Epoch [2/20], Step [30000/257353], Loss: 1.0588, Acc: 0.00
Epoch [2/20], Step [40000/257353], Loss: 1.2524, Acc: 0.00
Epoch [2/20], Step [50000/257353], Loss: 0.5515, Acc: 1.00
Epoch [2/20], Step [60000/257353], Loss: 0.5514, Acc: 1.00
Epoch [2/20], Step [70000/257353], Loss: 1.5514, Acc: 0.00
Epoch [2/20], Step [80000/257353], Loss: 1.4608, Acc: 0.00
Epoch [2/20], Step [90000/257353], Loss: 1.5511, Acc: 0.00
Epoch [2/20], Step [100000/257353], Loss: 1.5458, Acc: 0.00
Epoch [2/20], Step [110000/257353], Loss: 1.5514, Acc: 0.00
Epoch [2/20], Step [120000/257353], Loss: 0.5514, Acc: 1.00
Epoch [2/20], Step [130000/257353], Loss: 1.5513, Acc: 0.00
Epoch [2/20], Step [140000/257353], Loss: 0.5514, Acc: 1.00
Epoch [2/20], Step [150000/257353], Loss: 0.5530, Acc: 1.00
Epoch [2/20], Step [160000/257353], Loss: 0.5514, Acc: 1.00
Epoch [2/20], Step 

#####Resume Training

Load state dictionaries from checkpoint.

In [0]:
checkpoint = torch.load('checkpoint13.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch_chkpoint = checkpoint['epoch']
loss = checkpoint['loss']

In [0]:
for epoch in range(epoch_chkpoint + 1, epochs + 1):
  i = 0
  for data, label in zip(emb_train, lab_train):
    i += 1
    data = torch.from_numpy(data)
    bag_label = label
            
    if cuda:
      data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
                
    d = data.reshape(*list(data.shape), 1)
    one_hot = F.one_hot(torch.tensor(bag_label), 3)
    data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))
    

    optimizer.zero_grad()

    # Forward pass, backward pass, optimize
    prob, hat, A = model.forward(data)
            
    loss_size = loss(prob, torch.argmax(bag_label,1))
    loss_size.backward()
    optimizer.step()

    running_loss = loss_size.data
    total_train_loss += loss_size.data
    accuracy = (torch.argmax(bag_label,1).to(torch.float) == hat.squeeze()).float().sum()
    
    if (i+1) % 10000 == 0:
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Acc: {:.2f}' 
               .format(epoch, epochs, i+1, len(emb_train), loss_size.item(), accuracy.item()))

        # ================================================================== #
        #                        Tensorboard Logging                         #
        # ================================================================== #

       # 1. Log scalar values (scalar summary)
        info = { 'loss': loss_size.item(), 'accuracy': accuracy.item() }

        for tag, value in info.items():
            writer.add_scalar(tag, value, i+1)

        # 2. Log values and gradients of the parameters (histogram summary)
        for tag, value in model.named_parameters():
            tag = tag.replace('.', '/')
            writer.add_histogram(tag, value.data.cpu().numpy(), i+1)
            writer.add_histogram(tag+'/grad', value.grad.data.cpu().numpy(), i+1)
            
            
  torch.save({
      'epoch': epoch,
      'model_state_dict': model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
      'loss': loss,
       }, model_chk+str(epoch)+'.pt')
  
  validate(emb_val, lab_val) # Validation after each epoch

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch [14/20], Step [10000/257353], Loss: 1.5469, Acc: 0.00
Epoch [14/20], Step [20000/257353], Loss: 0.5558, Acc: 1.00
Epoch [14/20], Step [30000/257353], Loss: 1.4972, Acc: 0.00
Epoch [14/20], Step [40000/257353], Loss: 1.5509, Acc: 0.00
Epoch [14/20], Step [50000/257353], Loss: 0.5515, Acc: 1.00
Epoch [14/20], Step [60000/257353], Loss: 0.5514, Acc: 1.00
Epoch [14/20], Step [70000/257353], Loss: 1.5514, Acc: 0.00
Epoch [14/20], Step [80000/257353], Loss: 0.9502, Acc: 1.00
Epoch [14/20], Step [90000/257353], Loss: 1.5513, Acc: 0.00
Epoch [14/20], Step [100000/257353], Loss: 1.4977, Acc: 0.00
Epoch [14/20], Step [110000/257353], Loss: 1.5514, Acc: 0.00
Epoch [14/20], Step [120000/257353], Loss: 0.5514, Acc: 1.00
Epoch [14/20], Step [130000/257353], Loss: 0.5611, Acc: 1.00
Epoch [14/20], Step [140000/257353], Loss: 0.5514, Acc: 1.00
Epoch [14/20], Step [150000/257353], Loss: 0.5586, Acc: 1.00
Epoch [14/20], Step [160000/257353], Loss: 0.5514, Acc: 1.00
Epoch [14/20], Step [170000/25735

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':


Validation Accuracy :  73.622
Epoch [15/20], Step [10000/257353], Loss: 1.5417, Acc: 0.00
Epoch [15/20], Step [20000/257353], Loss: 0.5761, Acc: 1.00
Epoch [15/20], Step [30000/257353], Loss: 1.5509, Acc: 0.00
Epoch [15/20], Step [40000/257353], Loss: 0.5672, Acc: 1.00
Epoch [15/20], Step [50000/257353], Loss: 0.5515, Acc: 1.00
Epoch [15/20], Step [60000/257353], Loss: 0.5514, Acc: 1.00
Epoch [15/20], Step [70000/257353], Loss: 1.5511, Acc: 0.00
Epoch [15/20], Step [80000/257353], Loss: 1.3036, Acc: 0.00
Epoch [15/20], Step [90000/257353], Loss: 1.5514, Acc: 0.00
Epoch [15/20], Step [100000/257353], Loss: 1.5484, Acc: 0.00
Epoch [15/20], Step [110000/257353], Loss: 1.5514, Acc: 0.00
Epoch [15/20], Step [120000/257353], Loss: 0.5514, Acc: 1.00
Epoch [15/20], Step [130000/257353], Loss: 0.5525, Acc: 1.00
Epoch [15/20], Step [140000/257353], Loss: 0.5514, Acc: 1.00
Epoch [15/20], Step [150000/257353], Loss: 0.6168, Acc: 1.00
Epoch [15/20], Step [160000/257353], Loss: 0.5514, Acc: 1.00
Epo

####Test

In [0]:
i = 0
correct = 0
y_target = list()
y_pred = list()
for data, label in zip(emb_test, lab_test):
  data = torch.from_numpy(data)
  bag_label = label
  if cuda:
    data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
  d = data.reshape(*list(data.shape), 1)
  one_hot = F.one_hot(torch.tensor(bag_label), 3)
  data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))
  
  prob, hat, A = model.forward(data)
  y_target.append(int(label))
  y_pred.append(hat.int().item())

  if int(hat.item()) == label:
    correct += 1
    
    
print((correct/len(lab_test)) * 100)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


74.268


####Validate

In [0]:
def validate(emb_val, lab_val):
  i = 0
  correct = 0
  y_target = list()
  y_pred = list()
  for data, label in zip(emb_val, lab_val):
    data = torch.from_numpy(data)
    bag_label = label
    if cuda:
      data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
    d = data.reshape(*list(data.shape), 1)
    one_hot = F.one_hot(torch.tensor(bag_label), 3)
    data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))
    
    prob, hat, A = model.forward(data)
    y_target.append(int(label))
    y_pred.append(hat.int().item())
    if int(hat.item()) == label:
      correct += 1


  print('Validation Accuracy : ', (correct/len(lab_val)) * 100)

#### Analyze

In [0]:
i = 0
correct = 0
for data, label in zip(emb_test, lab_test):
  data = torch.from_numpy(data)
  bag_label = label
  if cuda:
    data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
  d = data.reshape(*list(data.shape), 1)
  one_hot = F.one_hot(torch.tensor(bag_label), 3)
  data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))

  prob, hat, A = model.forward(data)
  print(i)
  print('Prob :', prob, '\nPredicted : ' , hat, '\nAttention : ',  A,'\nTarget : ', label, '\n*************************\n')
  
  i += 1
  if i == 50:
    break
  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


0
Prob : tensor([[9.9998e-01, 1.6217e-05, 2.7792e-07]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([0.], device='cuda:0') 
Attention :  tensor([[0.2658, 0.7342]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Target :  1 
*************************

1
Prob : tensor([[8.9767e-07, 7.7687e-01, 2.2313e-01]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([1.], device='cuda:0') 
Attention :  tensor([[0.1912, 0.1900, 0.1919, 0.1856, 0.1907, 0.0506]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Target :  1 
*************************

2
Prob : tensor([[1.5981e-02, 9.8402e-01, 2.8543e-14]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([1.], device='cuda:0') 
Attention :  tensor([[0.4251, 0.4119, 0.1630]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Target :  1 
*************************

3
Prob : tensor([[2.2411e-17, 5.3316e-08, 1.0000e+00]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  t

####Save

In [0]:
model_num = 0

In [0]:
PATH = '1dpool_model_state_CNN_20_Epochs'+str(model_num)+'.pt'
PATH0 = '1dpool_model__CNN_20_Epochs'+str(model_num)+'.pt'

torch.save(model.state_dict(), PATH)

torch.save(model, PATH0)

model_num += 1

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Attention. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


####Metrics

In [0]:
evaluate_model(y_target, y_pred)

Accuracy: 0.74268; 
F1-Score (Micro): 0.74268; 
F1-Score (Macro): 0.7417440937348956; 
F1-Score (Weighted): 0.7430335352945195


###Fine-Tuning

#### Load Organic Dataset

In [0]:
train = np.load('O_train_bag.npy')
test = np.load('O_test_bag.npy')
val = np.load('O_val_bag.npy')

In [0]:
emd_train = train[0]
lab_train = train[1]


emd_test = test[0]
lab_test = test[1]


emd_val = val[0]
lab_val = val[1]

In [0]:
model = torch.load('model__CNN_50_Epochs1.pt')

In [0]:
model.classifier

Sequential(
  (0): Linear(in_features=500, out_features=3, bias=True)
  (1): Softmax()
)

In [0]:
organic_model = Attention()

In [0]:
organic_model.load_state_dict(model.state_dict(), strict=False)

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [0]:
seed = 5

if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    print('GPU is ON!')

    
if torch.cuda.is_available():
    organic_model.cuda()

loss = torch.nn.CrossEntropyLoss()

cuda = True

running_loss = 0
total_train_loss = 0
i = 0
epochs = 50

GPU is ON!


In [0]:
optimizer = optim.Adam(organic_model.parameters(), lr=0.000005, betas=(0.9, 0.999), weight_decay=10e-5)

####Train

In [0]:
model_chk = './organic_checkpoint'

In [0]:
organic_model.train()

Attention(
  (feature_extractor_part1): Sequential(
    (0): Conv1d(768, 20, kernel_size=(1,), stride=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(20, 50, kernel_size=(1,), stride=(1,))
    (4): ReLU()
    (5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (feature_extractor_part2): Sequential(
    (0): Linear(in_features=50, out_features=500, bias=True)
    (1): ReLU()
  )
  (attention): Sequential(
    (0): Linear(in_features=500, out_features=128, bias=True)
    (1): Tanh()
    (2): Linear(in_features=128, out_features=1, bias=True)
  )
  (classifier): Sequential(
    (0): Linear(in_features=500, out_features=3, bias=True)
    (1): Softmax()
  )
)

In [0]:
for epoch in range(1, epochs + 1):
  i = 0
  for data, label in zip(emd_train, lab_train):
    i += 1
    data = torch.from_numpy(data)
    bag_label = label
            
    if cuda:
      data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
                
    d = data.reshape(*list(data.shape), 1)
    one_hot = F.one_hot(torch.tensor(bag_label), 3)
    data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))
    

    optimizer.zero_grad()

    # Forward pass, backward pass, optimize
    prob, hat, A = organic_model.forward(data)
            
    loss_size = loss(prob, torch.argmax(bag_label,1))
    loss_size.backward()
    optimizer.step()

    running_loss = loss_size.data
    total_train_loss += loss_size.data
    accuracy = (torch.argmax(bag_label,1).to(torch.float) == hat.squeeze()).float().sum()
    
    if (i+1) % 1000 == 0:
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Acc: {:.2f}' 
               .format(epoch, epochs, i+1, len(emd_train), loss_size.item(), accuracy.item()))

        # ================================================================== #
        #                        Tensorboard Logging                         #
        # ================================================================== #

       # 1. Log scalar values (scalar summary)
        info = { 'loss': loss_size.item(), 'accuracy': accuracy.item() }

        for tag, value in info.items():
            writer.add_scalar(tag, value, i+1)

        # 2. Log values and gradients of the parameters (histogram summary)
        for tag, value in model.named_parameters():
            tag = tag.replace('.', '/')
            writer.add_histogram(tag, value.data.cpu().numpy(), i+1)
            
            
  torch.save({
      'epoch': epoch,
      'model_state_dict': model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
      'loss': loss,
       }, model_chk+str(epoch)+'.pt')
  
  validate(emd_val, lab_val)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch [1/50], Step [1000/4687], Loss: 1.5408, Acc: 0.00
Epoch [1/50], Step [2000/4687], Loss: 1.4120, Acc: 0.00
Epoch [1/50], Step [3000/4687], Loss: 1.5514, Acc: 0.00
Epoch [1/50], Step [4000/4687], Loss: 1.5485, Acc: 0.00


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':


Validation Accuracy :  44.68599033816425
Epoch [2/50], Step [1000/4687], Loss: 1.5240, Acc: 0.00
Epoch [2/50], Step [2000/4687], Loss: 1.3986, Acc: 0.00
Epoch [2/50], Step [3000/4687], Loss: 1.5514, Acc: 0.00
Epoch [2/50], Step [4000/4687], Loss: 1.5489, Acc: 0.00
Validation Accuracy :  46.13526570048309
Epoch [3/50], Step [1000/4687], Loss: 1.4780, Acc: 0.00
Epoch [3/50], Step [2000/4687], Loss: 1.3856, Acc: 0.00
Epoch [3/50], Step [3000/4687], Loss: 1.5481, Acc: 0.00
Epoch [3/50], Step [4000/4687], Loss: 1.5491, Acc: 0.00
Validation Accuracy :  47.82608695652174
Epoch [4/50], Step [1000/4687], Loss: 1.4297, Acc: 0.00
Epoch [4/50], Step [2000/4687], Loss: 1.3703, Acc: 0.00
Epoch [4/50], Step [3000/4687], Loss: 1.4859, Acc: 0.00
Epoch [4/50], Step [4000/4687], Loss: 1.5486, Acc: 0.00
Validation Accuracy :  47.34299516908212
Epoch [5/50], Step [1000/4687], Loss: 1.3302, Acc: 0.00
Epoch [5/50], Step [2000/4687], Loss: 1.3626, Acc: 0.00
Epoch [5/50], Step [3000/4687], Loss: 0.9700, Acc: 0

####Validate

In [0]:
def validate(emb_val, lab_val):
  i = 0
  correct = 0
  y_target = list()
  y_pred = list()
  for data, label in zip(emb_val, lab_val):
    data = torch.from_numpy(data)
    bag_label = label
    if cuda:
      data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
    d = data.reshape(*list(data.shape), 1)
    one_hot = F.one_hot(torch.tensor(bag_label), 3)
    data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))
    
    prob, hat, A = organic_model.forward(data)
    y_target.append(int(label))
    y_pred.append(hat.int().item())
    if int(hat.item()) == label:
      correct += 1


  print('Validation Accuracy : ', (correct/len(lab_val)) * 100)

####Test

In [0]:
i = 0
correct = 0
y_target = list()
y_pred = list()
for data, label in zip(emd_test, lab_test):
  data = torch.from_numpy(data)
  bag_label = label
  if cuda:
    data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
  d = data.reshape(*list(data.shape), 1)
  one_hot = F.one_hot(torch.tensor(bag_label), 3)
  data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))
  
  prob, hat, A = organic_model.forward(data)
  y_target.append(int(label))
  y_pred.append(hat.int().item())

  if int(hat.item()) == label:
    correct += 1
    
    
print((correct/len(lab_test)) * 100)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


55.00000000000001


####Analyze

In [0]:
i = 0
correct = 0
for data, label in zip(emd_test, lab_test):
  data = torch.from_numpy(data)
  bag_label = label
  if cuda:
    data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
  d = data.reshape(*list(data.shape), 1)
  one_hot = F.one_hot(torch.tensor(bag_label), 3)
  data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))

  prob, hat, A = organic_model.forward(data)
  print(i)
  print('Prob :', prob, '\nPredicted : ' , hat, '\nAttention : ',  A,'\nTarget : ', label, '\n*************************\n')
  
  i += 1
  if i == 50:
    break

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


0
Prob : tensor([[0.2580, 0.7337, 0.0083]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([1.], device='cuda:0') 
Attention :  tensor([[1.]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Target :  1 
*************************

1
Prob : tensor([[9.8988e-01, 1.0116e-02, 6.0260e-12]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([0.], device='cuda:0') 
Attention :  tensor([[1.]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Target :  1 
*************************

2
Prob : tensor([[7.9797e-02, 9.2004e-01, 1.6028e-04]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([1.], device='cuda:0') 
Attention :  tensor([[1.]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Target :  1 
*************************

3
Prob : tensor([[0.0155, 0.0125, 0.9720]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([2.], device='cuda:0') 
Attention :  tensor([[1.]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Target :  1 
*****

####Save

In [0]:
model_num = 0

In [0]:
PATH = '_organicTF50epoch'+str(model_num)+'.pt'
PATH0 = '_organicTF50_epoch_model'+str(model_num)+'.pt'

torch.save(model.state_dict(), PATH)

torch.save(model, PATH0)

model_num += 1

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Attention. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


####Evaluate

In [0]:
evaluate_model(y_target, y_pred)

Accuracy: 0.55; 
F1-Score (Micro): 0.55; 
F1-Score (Macro): 0.5316447252940143; 
F1-Score (Weighted): 0.5465790919706024


### Train more epochs

In [0]:
checkpoint = torch.load('organic_checkpoint50.pt')
organic_model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch_chkpoint = checkDatasrtpoint['epoch']
loss = checkpoint['loss']

In [0]:
epochs = 100

#####Resume Training

In [0]:
for epoch in range(epoch_chkpoint + 1, epochs + 1):
  i = 0
  for data, label in zip(emd_train, lab_train):
    i += 1
    data = torch.from_numpy(data)
    bag_label = label
            
    if cuda:
      data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
                
    d = data.reshape(*list(data.shape), 1)
    one_hot = F.one_hot(torch.tensor(bag_label), 3)
    data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))
    

    optimizer.zero_grad()

    # Forward pass, backward pass, optimize
    prob, hat, A = organic_model.forward(data)
            
    loss_size = loss(prob, torch.argmax(bag_label,1))
    loss_size.backward()
    optimizer.step()

    running_loss = loss_size.data
    total_train_loss += loss_size.data
    accuracy = (torch.argmax(bag_label,1).to(torch.float) == hat.squeeze()).float().sum()
    
    if (i+1) % 1000 == 0:
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Acc: {:.2f}' 
               .format(epoch, epochs, i+1, len(emd_train), loss_size.item(), accuracy.item()))

        # ================================================================== #
        #                        Tensorboard Logging                         #
        # ================================================================== #

       # 1. Log scalar values (scalar summary)
        info = { 'loss': loss_size.item(), 'accuracy': accuracy.item() }

        for tag, value in info.items():
            writer.add_scalar(tag, value, i+1)

        # 2. Log values and gradients of the parameters (histogram summary)
        for tag, value in model.named_parameters():
            tag = tag.replace('.', '/')
            writer.add_histogram(tag, value.data.cpu().numpy(), i+1)
            writer.add_histogram(tag+'/grad', value.grad.data.cpu().numpy(), i+1)
            
            
  torch.save({
      'epoch': epoch,
      'model_state_dict': model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
      'loss': loss,
       }, model_chk+str(epoch)+'.pt')
  
  validate(emd_val, lab_val)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch [51/100], Step [1000/4687], Loss: 1.5174, Acc: 0.00
Epoch [51/100], Step [2000/4687], Loss: 1.4024, Acc: 0.00
Epoch [51/100], Step [3000/4687], Loss: 1.5514, Acc: 0.00
Epoch [51/100], Step [4000/4687], Loss: 1.5486, Acc: 0.00


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':


Validation Accuracy :  46.13526570048309
Epoch [52/100], Step [1000/4687], Loss: 1.4780, Acc: 0.00
Epoch [52/100], Step [2000/4687], Loss: 1.3898, Acc: 0.00
Epoch [52/100], Step [3000/4687], Loss: 1.5482, Acc: 0.00
Epoch [52/100], Step [4000/4687], Loss: 1.5489, Acc: 0.00
Validation Accuracy :  47.58454106280193
Epoch [53/100], Step [1000/4687], Loss: 1.4371, Acc: 0.00
Epoch [53/100], Step [2000/4687], Loss: 1.3765, Acc: 0.00
Epoch [53/100], Step [3000/4687], Loss: 1.4871, Acc: 0.00
Epoch [53/100], Step [4000/4687], Loss: 1.5485, Acc: 0.00
Validation Accuracy :  47.10144927536232
Epoch [54/100], Step [1000/4687], Loss: 1.3539, Acc: 0.00
Epoch [54/100], Step [2000/4687], Loss: 1.3650, Acc: 0.00
Epoch [54/100], Step [3000/4687], Loss: 0.9627, Acc: 0.00
Epoch [54/100], Step [4000/4687], Loss: 1.5479, Acc: 0.00
Validation Accuracy :  46.3768115942029
Epoch [55/100], Step [1000/4687], Loss: 1.2495, Acc: 0.00
Epoch [55/100], Step [2000/4687], Loss: 1.3620, Acc: 0.00
Epoch [55/100], Step [300

#####Test

In [0]:
i = 0
correct = 0
y_target = list()
y_pred = list()
for data, label in zip(emd_test, lab_test):
  data = torch.from_numpy(data)
  bag_label = label
  if cuda:
    data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
  d = data.reshape(*list(data.shape), 1)
  one_hot = F.one_hot(torch.tensor(bag_label), 3)
  data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))
 
  prob, hat, A = organic_model.forward(data)
  y_target.append(int(label))
  y_pred.append(hat.int().item())

  if int(hat.item()) == label:
    correct += 1
    
    
print((correct/len(lab_test)) * 100)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


55.434782608695656


#####Evaluate

In [0]:
evaluate_model(y_target, y_pred)

Accuracy: 0.5543478260869565; 
F1-Score (Micro): 0.5543478260869565; 
F1-Score (Macro): 0.536172742635288; 
F1-Score (Weighted): 0.5508453844164247


##More number of filters

We thought let us try to increase the number of filters, if it has any effect on the performance.

####Validation

In [0]:
def validate(emb_val, lab_val):
  i = 0
  correct = 0
  y_target = list()
  y_pred = list()
  for data, label in zip(emb_val, lab_val):
    data = torch.from_numpy(data)
    bag_label = label
    if cuda:
      data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
    d = data.reshape(*list(data.shape), 1)
    one_hot = F.one_hot(torch.tensor(bag_label), 3)
    data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))
    
    prEvaluateob, hat, A = model.forward(data)
    y_target.append(int(label))
    y_pred.append(hat.int().item())
    if int(hat.item()) == label:
      correct += 1


  print('Validation Accuracy : ', (correct/len(lab_val)) * 100)

#### Model Class

In [0]:
class Attention(nn.Module):
    def __init__(self):
        super(Attention, self).__init__()
        self.L = 500
        self.D = 128
        self.K = 1

        self.feature_extractor_part1 = nn.Sequential(
            nn.Conv1d(768, 512, kernel_size=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv1d(256, 256, kernel_size=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2)
        )

        self.feature_extractor_part2 = nn.Sequential(
            nn.Linear(128 * 1, self.L),
            nn.ReLU(),
        )

        self.attention = nn.Sequential(
            nn.Linear(self.L, self.D),
            nn.Tanh(),
            nn.Linear(self.D, self.K)
        )

        self.classifier = nn.Sequential(
            nn.Linear(self.L*self.K, 3),
            nn.Softmax()
        )

    def forward(self, x):
        H = self.feature_extractor_part1(x)
        H = H.view(-1, 128 * 1)
        H = self.feature_extractor_part2(H)  # NxL

        A = self.attention(H)  # NxK
        A = torch.transpose(A, 1, 0)  # KxN
        A = F.softmax(A, dim=1)  # softmax over N

        M = torch.mm(A, H)  # KxL

        Y_prob = self.classifier(M)
        
        Y_hat = torch.argmax(Y_prob, dim=1).float()

        return Y_prob, Y_hat, A

In [0]:
seed = 5

if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    print('GPU is ON!')


GPU is ON!


In [0]:
model = Attention()
if torch.cuda.is_available():
    model.cuda()

print('Init Model')

Init Model


In [0]:
loss = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005, betas=(0.9, 0.999), weight_decay=10e-5)

In [0]:
cuda = True

In [0]:
model_chk = './checkpoint'

In [0]:
running_loss = 0
total_train_loss = 0
i = 0
epochs = 20

In [0]:
model.train()

Attention(
  (feature_extractor_part1): Sequential(
    (0): Conv1d(768, 512, kernel_size=(1,), stride=(1,))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (feature_extractor_part2): Sequential(
    (0): Linear(in_features=128, out_features=500, bias=True)
    (1): ReLU()
  )
  (attention): Sequential(
    (0): Linear(in_features=500, out_features=128, bias=True)
    (1): Tanh()
    (2): Linear(in_features=128, out_features=1, bias=True)
  )
  (classifier): Sequential(
    (0): Linear(in_features=500, out_features=3, bias=True)
    (1): Softmax()
  )
)

####Train

In [0]:
for epoch in range(1, epochs + 1):
  i = 0
  for data, label in zip(emb_train, lab_train):
    i += 1
    data = torch.from_numpy(data)
    bag_label = label
            
    if cuda:
      data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
                
    d = data.reshape(*list(data.shape), 1)
    one_hot = F.one_hot(torch.tensor(bag_label), 3)
    data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))
    

    optimizer.zero_grad()

    # Forward pass, backward pass, optimize
    prob, hat, A = model.forward(data)
            
    loss_size = loss(prob, torch.argmax(bag_label,1))
    loss_size.backward()
    optimizer.step()

    running_loss = loss_size.data
    total_train_loss += loss_size.data
    accuracy = (torch.argmax(bag_label,1).to(torch.float) == hat.squeeze()).float().sum()
    
    if (i+1) % 10000 == 0:
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Acc: {:.2f}' 
               .format(epoch, epochs, i+1, len(emb_train), loss_size.item(), accuracy.item()))

        # ================================================================== #
        #                        Tensorboard Logging                         #
        # ================================================================== #

       # 1. Log scalar values (scalar summary)
        info = { 'loss': loss_size.item(), 'accuracy': accuracy.item() }

        for tag, value in info.items():
            writer.add_scalar(tag, value, i+1)

        # 2. Log values and gradients of the parameters (histogram summary)
        for tag, value in model.named_parameters():
            tag = tag.replace('.', '/')
            writer.add_histogram(tag, value.data.cpu().numpy(), i+1)
            writer.add_histogram(tag+'/grad', value.grad.data.cpu().numpy(), i+1)
            
            
  torch.save({
      'epoch': epoch,
      'model_state_dict': model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
      'loss': loss,
       }, model_chk+str(epoch)+'.pt')
  
  validate(emb_val, lab_val)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch [1/20], Step [10000/257353], Loss: 1.4633, Acc: 0.00
Epoch [1/20], Step [20000/257353], Loss: 0.8926, Acc: 1.00
Epoch [1/20], Step [30000/257353], Loss: 1.5143, Acc: 0.00
Epoch [1/20], Step [40000/257353], Loss: 0.5514, Acc: 1.00
Epoch [1/20], Step [50000/257353], Loss: 0.5549, Acc: 1.00
Epoch [1/20], Step [60000/257353], Loss: 0.5516, Acc: 1.00
Epoch [1/20], Step [70000/257353], Loss: 1.4245, Acc: 0.00
Epoch [1/20], Step [80000/257353], Loss: 0.5522, Acc: 1.00
Epoch [1/20], Step [90000/257353], Loss: 0.5514, Acc: 1.00
Epoch [1/20], Step [100000/257353], Loss: 0.5528, Acc: 1.00
Epoch [1/20], Step [110000/257353], Loss: 0.5761, Acc: 1.00
Epoch [1/20], Step [120000/257353], Loss: 0.5516, Acc: 1.00
Epoch [1/20], Step [130000/257353], Loss: 1.5428, Acc: 0.00
Epoch [1/20], Step [140000/257353], Loss: 0.5514, Acc: 1.00
Epoch [1/20], Step [150000/257353], Loss: 0.5514, Acc: 1.00
Epoch [1/20], Step [160000/257353], Loss: 0.8423, Acc: 1.00
Epoch [1/20], Step [170000/257353], Loss: 1.5214,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':


Validation Accuracy :  69.44200000000001
Epoch [2/20], Step [10000/257353], Loss: 1.5428, Acc: 0.00
Epoch [2/20], Step [20000/257353], Loss: 1.2129, Acc: 0.00
Epoch [2/20], Step [30000/257353], Loss: 1.5301, Acc: 0.00
Epoch [2/20], Step [40000/257353], Loss: 0.5514, Acc: 1.00
Epoch [2/20], Step [50000/257353], Loss: 0.5531, Acc: 1.00
Epoch [2/20], Step [60000/257353], Loss: 0.5519, Acc: 1.00
Epoch [2/20], Step [70000/257353], Loss: 1.3407, Acc: 0.00
Epoch [2/20], Step [80000/257353], Loss: 0.5698, Acc: 1.00
Epoch [2/20], Step [90000/257353], Loss: 0.5514, Acc: 1.00
Epoch [2/20], Step [100000/257353], Loss: 0.5516, Acc: 1.00
Epoch [2/20], Step [110000/257353], Loss: 0.5544, Acc: 1.00
Epoch [2/20], Step [120000/257353], Loss: 0.5514, Acc: 1.00
Epoch [2/20], Step [130000/257353], Loss: 1.3281, Acc: 0.00
Epoch [2/20], Step [140000/257353], Loss: 0.5514, Acc: 1.00
Epoch [2/20], Step [150000/257353], Loss: 0.5514, Acc: 1.00
Epoch [2/20], Step [160000/257353], Loss: 1.2681, Acc: 0.00
Epoch [2

####Test

In [0]:
i = 0
correct = 0
y_target = list()
y_pred = list()
for data, label in zip(emb_test, lab_test):
  data = torch.from_numpy(data)
  bag_label = label
  if cuda:
    data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
  d = data.reshape(*list(data.shape), 1)
  one_hot = F.one_hot(torch.tensor(bag_label), 3)
  data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))
  
  prob, hat, A = model.forward(data)
  y_target.append(int(label))
  y_pred.append(hat.int().item())
 
  if int(hat.item()) == label:
    correct += 1
    
    
print((correct/len(lab_test)) * 100)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


72.038


#### Analyze

In [0]:
i = 0
correct = 0
for data, label in zip(emb_test, lab_test):
  data = torch.from_numpy(data)
  bag_label = label
  if cuda:
    data, bag_label = data.cuda(), torch.tensor(bag_label).cuda()
  d = data.reshape(*list(data.shape), 1)
  one_hot = F.one_hot(torch.tensor(bag_label), 3)
  data, bag_label = Variable(torch.tensor(d)), Variable(one_hot.unsqueeze(0))
 
  prob, hat, A = model.forward(data)
  print(i)
  print('Prob :', prob, '\nPredicted : ' , hat, '\nAttention : ',  A,'\nTarget : ', label, '\n*************************\n')
  
  i += 1
  if i == 50:
    break
  

0
Prob : tensor([[0.9899, 0.0054, 0.0046]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([0.], device='cuda:0') 
Attention :  tensor([[0.4930, 0.5070]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Target :  1 
*************************

1
Prob : tensor([[1.0658e-07, 8.7821e-01, 1.2179e-01]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([1.], device='cuda:0') 
Attention :  tensor([[0.1680, 0.1678, 0.1668, 0.1674, 0.1680, 0.1620]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Target :  1 
*************************

2
Prob : tensor([[2.7709e-01, 7.2291e-01, 4.4433e-16]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([1.], device='cuda:0') 
Attention :  tensor([[0.3391, 0.3399, 0.3210]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Target :  1 
*************************

3
Prob : tensor([[7.2518e-16, 4.1519e-09, 1.0000e+00]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([2.], device=

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


tensor([[2.7197e-12, 5.2282e-06, 9.9999e-01]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([2.], device='cuda:0') 
Attention :  tensor([[0.3119, 0.3424, 0.3456]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Target :  2 
*************************

29
Prob : tensor([[1.0000e+00, 9.8395e-09, 1.3061e-21]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([0.], device='cuda:0') 
Attention :  tensor([[0.1721, 0.1721, 0.1715, 0.1408, 0.1713, 0.1722]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Target :  0 
*************************

30
Prob : tensor([[9.9994e-01, 5.9221e-05, 3.0602e-13]], device='cuda:0',
       grad_fn=<SoftmaxBackward>) 
Predicted :  tensor([0.], device='cuda:0') 
Attention :  tensor([[0.0679, 0.0846, 0.0846, 0.0804, 0.0856, 0.0834, 0.0856, 0.0856, 0.0856,
         0.0856, 0.0855, 0.0855]], device='cuda:0', grad_fn=<SoftmaxBackward>) 
Target :  0 
*************************

31
Prob : tensor([[7.7147e-25, 9.1365e-1

####Save

In [0]:
model_num = 0

In [0]:
PATH = 'model_state_CNN512'+str(model_num)
PATH0 = 'model_CNN512.pt'

torch.save(model.state_dict(), PATH)

torch.save(model, PATH0)

model_num += 1

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Attention. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


####Metrics

In [0]:
evaluate_model(y_target, y_pred)

Accuracy: 0.72038; 
F1-Score (Micro): 0.72038; 
F1-Score (Macro): 0.7145053553637407; 
F1-Score (Weighted): 0.71586851750882


We could also try to change the size the size of filters, and other hyperparameters. Since all the variations we tried resulted into similar performances we skipped it for now. But it is definetaly something for us to try out in *Future Scopes*